In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from pyproj import Proj
from scipy import interpolate as intp
import seaborn as sns
import warnings
import scipy.stats as stats
import numpy as np
import logging
from scipy.signal import gaussian, convolve
warnings.filterwarnings('ignore')

### IGBP land cover classification system. 
1. Evergreen needleleaf forests
2. Evergreen broadleaf forests
3. Deciduous needleleaf forests
4. Deciduous broadleaf forests
5. Mixed forests
6. Shrublands
7. Grasslands 
8. Croplands 
9. Vine

### Columns of compiled dataset:
"Dataset ID", "Dataset Name", "Site ID", "Sample date", "English Name", "Latin Name",
"PFT", "Latitude", "Longitude", "Chla+b", "Ccar", "EWT", "LMA", "Foreoptic type",
"Instruments", "Reference","Link to original dataset","DOI"

- Chla+b: Chlorophyll_a+b (µg/cm²);
- Ccar: Carotenoid (µg/cm²), 
- EWT: Equivalent Water Thickness (g/m²), 
- LMA: Leaf mass per area (g/m²)


### FWHM resampling method.

In [2]:
class BandInfo:
    '''A BandInfo object characterizes the spectral bands associated with an
    image. All BandInfo member variables are optional.  For *N* bands, all
    members of type <list> will have length *N* and contain float values.

    =================   =====================================   =======
        Member                  Description                     Default
    =================   =====================================   =======
    centers             List of band centers                    None
    bandwidths          List of band FWHM values                None
    centers_stdevs      List of std devs of band centers        None
    bandwidth_stdevs    List of std devs of bands FWHMs         None
    band_quantity       Image data type (e.g., "reflectance")   ""
    band_unit           Band unit (e.g., "nanometer")           ""
    =================   =====================================   =======
    '''
    def __init__(self):
        self.centers = None
        self.bandwidths = None
        self.centers_stdevs = None
        self.bandwidth_stdevs = None
        self.band_quantity = None
        self.band_unit = None

def erf_local(x):
    # save the sign of x
    sign = 1 if x >= 0 else -1
    x = abs(x)

    # constants
    a1 =  0.254829592
    a2 = -0.284496736
    a3 =  1.421413741
    a4 = -1.453152027
    a5 =  1.061405429
    p  =  0.3275911

    # A&S formula 7.1.26
    t = 1.0/(1.0 + p*x)
    y = 1.0 - (((((a5*t + a4)*t) + a3)*t + a2)*t + a1)*t*math.exp(-x*x)
    return sign*y # erf(-x) = -erf(x)

try:
    from math import erf
except:
    try:
        from scipy.special import erf
    except:
        erf = erf_local

def erfc(z):
    '''Complement of the error function.'''
    return 1.0 - erf(z)

def normal_cdf(x):
    '''CDF of the normal distribution.'''
    sqrt2 = 1.4142135623730951
    return 0.5 * erfc(-x / sqrt2)

def normal_integral(a, b):
    '''Integral of the normal distribution from a to b.'''
    return normal_cdf(b) - normal_cdf(a)

def ranges_overlap(R1, R2):
    '''Returns True if there is overlap between ranges of pairs R1 and R2.'''
    if (R1[0] < R2[0] and R1[1] < R2[0]) or \
       (R1[0] > R2[1] and R1[1] > R2[1]):
        return False
    return True

def overlap(R1, R2):
    '''Returns (min, max) of overlap between the ranges of pairs R1 and R2.'''
    return (max(R1[0], R2[0]), min(R1[1], R2[1]))

def normal(mean, stdev, x):
    sqrt_2pi = 2.5066282746310002
    return math.exp(-((x - mean) / stdev)**2 / 2.0) / (sqrt_2pi * stdev)

def build_fwhm(centers):
    '''Returns FWHM list, assuming FWHM is midway between adjacent bands.
    '''
    fwhm = [0] * len(centers)
    fwhm[0] = centers[1] - centers[0]
    fwhm[-1] = centers[-1] - centers[-2]
    for i in range(1, len(centers) - 1):
        fwhm[i] = (centers[i + 1] - centers[i - 1]) / 2.0
    return fwhm

def create_resampling_matrix(centers1, fwhm1, centers2, fwhm2):
    '''
    Returns a resampling matrix to convert spectra from one band discretization
    to another.  Arguments are the band centers and full-width half maximum
    spectral response for the original and new band discretizations.
    '''
    logger = logging.getLogger('spectral')

    sqrt_8log2 = 2.3548200450309493

    N1 = len(centers1)
    N2 = len(centers2)
    bounds1 = [[centers1[i] - fwhm1[i] / 2.0, centers1[i] + fwhm1[i] /
                2.0] for i in range(N1)]
    bounds2 = [[centers2[i] - fwhm2[i] / 2.0, centers2[i] + fwhm2[i] /
                2.0] for i in range(N2)]

    M = np.zeros([N2, N1])

    jStart = 0
    nan = float('nan')
    for i in range(N2):
        stdev = fwhm2[i] / sqrt_8log2
        j = jStart

        # Find the first original band that overlaps the new band
        while j < N1 and bounds1[j][1] < bounds2[i][0]:
            j += 1

        if j == N1:
            logger.info(('No overlap for target band %d (%f / %f)' % (
                i, centers2[i], fwhm2[i])))
            M[i, 0] = nan
            continue

        matches = []

        # Get indices for all original bands that overlap the new band
        while j < N1 and bounds1[j][0] < bounds2[i][1]:
            if ranges_overlap(bounds1[j], bounds2[i]):
                matches.append(j)
            j += 1

        # Put NaN in first element of any row that doesn't produce a band in
        # the new schema.
        if len(matches) == 0:
            logger.info('No overlap for target band %d (%f / %f)',
                         i, centers2[i], fwhm2[i])
            M[i, 0] = nan
            continue

        # Determine the weights for the original bands that overlap the new
        # band. There may be multiple bands that overlap or even just a single
        # band that only partially overlaps.  Weights are normoalized so either
        # case can be handled.

        overlaps = [overlap(bounds1[k], bounds2[i]) for k in matches]
        contribs = np.zeros(len(matches))
        A = 0.
        for k in range(len(matches)):
            #endNorms = [normal(centers2[i], stdev, x) for x in overlaps[k]]
            #dA = (overlaps[k][1] - overlaps[k][0]) * sum(endNorms) / 2.0
            (a, b) = [(x - centers2[i]) / stdev for x in overlaps[k]]
            dA = normal_integral(a, b)
            contribs[k] = dA
            A += dA
        contribs = contribs / A
        for k in range(len(matches)):
            M[i, matches[k]] = contribs[k]
    return M

class BandResampler:
    '''A callable object for resampling spectra between band discretizations.

    A source band will contribute to any destination band where there is
    overlap between the FWHM of the two bands.  If there is an overlap, an
    integral is performed over the region of overlap assuming the source band
    data value is constant over its FWHM (since we do not know the true
    spectral load over the source band) and the destination band has a Gaussian
    response function. Any target bands that do not have any overlapping source
    bands will contain NaN as the resampled band value.

    If bandwidths are not specified for source or destination bands, the bands
    are assumed to have FWHM values that span half the distance to the adjacent
    bands.
    '''
    def __init__(self, centers1, centers2, fwhm1=None, fwhm2=None):
        '''BandResampler constructor.

        Usage:

            resampler = BandResampler(bandInfo1, bandInfo2)

            resampler = BandResampler(centers1, centers2, [fwhm1 = None [, fwhm2 = None]])

        Arguments:

            `bandInfo1` (:class:`~spectral.BandInfo`):

                Discretization of the source bands.

            `bandInfo2` (:class:`~spectral.BandInfo`):

                Discretization of the destination bands.

            `centers1` (list):

                floats defining center values of source bands.

            `centers2` (list):

                floats defining center values of destination bands.

            `fwhm1` (list):

                Optional list defining FWHM values of source bands.

            `fwhm2` (list):

                Optional list defining FWHM values of destination bands.

        Returns:

            A callable BandResampler object that takes a spectrum corresponding
            to the source bands and returns the spectrum resampled to the
            destination bands.

        If bandwidths are not specified, the associated bands are assumed to
        have FWHM values that span half the distance to the adjacent bands.
        '''
        if isinstance(centers1, BandInfo):
            fwhm1 = centers1.bandwidths
            centers1 = centers1.centers
        if isinstance(centers2, BandInfo):
            fwhm2 = centers2.bandwidths
            centers2 = centers2.centers
        if fwhm1 is None:
            fwhm1 = build_fwhm(centers1)
        if fwhm2 is None:
            fwhm2 = build_fwhm(centers2)
        self.matrix = create_resampling_matrix(
            centers1, fwhm1, centers2, fwhm2)

    def __call__(self, spectrum):
        '''Takes a source spectrum as input and returns a resampled spectrum.

        Arguments:

            `spectrum` (list or :class:`numpy.ndarray`):

                list or vector of values to be resampled.  Must have same
                length as the source band discretiation used to created the
                resampler.

        Returns:

            A resampled rank-1 :class:`numpy.ndarray` with length corresponding
            to the destination band discretization used to create the resampler.

        Any target bands that do not have at lease one overlapping source band
        will contain `float('nan')` as the resampled band value.'''
        return np.dot(self.matrix, spectrum)
def do_resample(data):
    centers1 = data.columns.astype(float)
    centers2 = np.arange(450,2401,10)
    fwhm1 = build_fwhm(centers1)
    fwhm2 = build_fwhm(centers2)
    resampler = BandResampler(centers1, centers2, fwhm1, fwhm2)
    start_var = True
    for i in range(len(data)):
        resampled_spectra = pd.DataFrame(resampler(data.iloc[i])).T
        resampled_spectra.columns = centers2.astype(int).astype(str)
        if start_var:
            new_data = resampled_spectra
            start_var = False
        else:
            new_data = pd.concat([new_data,resampled_spectra],axis = 0)
    new_data.reset_index(drop = True, inplace = True)
    return new_data

### 1. Lopex93

In [3]:
ds1_reflectance = pd.read_csv('../1_datasets/0_original/1_Lopex93/lopex1993reflectance.csv')
ds1_traits = pd.read_csv('../1_datasets/0_original/1_Lopex93/lopex1993metadata.csv')

ds1_traits['Latitude'] = 45.8054608
ds1_traits['Longitude'] = 8.6298954
ds1_pft = ['Grasslands','Grasslands','Vine','Deciduous broadleaf forests','Croplands','Shrublands','Deciduous broadleaf forests',
           'Croplands','Croplands','Croplands','Deciduous broadleaf forests','Shrublands','Deciduous broadleaf forests',
           'Deciduous broadleaf forests','Croplands','Deciduous broadleaf forests','Deciduous broadleaf forests','Croplands','Grasslands',
           'Croplands','Grasslands','Croplands','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests',
           'Croplands','Shrublands','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Grasslands',
           'Deciduous broadleaf forests','Shrublands','Shrublands','Grasslands','Deciduous broadleaf forests','Deciduous broadleaf forests',
           'Deciduous broadleaf forests','Croplands','Grasslands','Vine','Deciduous broadleaf forests','Deciduous broadleaf forests',
           'Grasslands','Deciduous broadleaf forests','Croplands']

for i in range(len(ds1_traits['English Name'].unique())):
    ds1_traits.loc[ds1_traits['English Name'] == ds1_traits['English Name'].unique()[i], 'PFT'] = ds1_pft[i]
    
ds1_traits.rename(columns={'Chlorophyll_a+b (µg/cm²)':'Chla+b','Carotenoid (µg/cm²)':'Ccar',
                           'Equivalent Water Thickness (g/cm²)':'EWT','Leaf mass per area (g/cm²)':'LMA'},inplace = True)

ds1_traits['Sample date'] = '1993'
ds1_traits['Dataset ID'] = 'Dataset#1'
ds1_traits['Site ID'] = 'Site#1'
ds1_traits['Dataset Name'] = 'Lopex93'
ds1_traits["Foreoptic type"] = "Integrating sphere"
ds1_traits["Instruments"] = "Lambda-19 Spectrophotometer"
ds1_traits["Reference"] = "(Hosgood et al., 1994)"
ds1_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-optical-properties-experiment-database--lopex93-"
ds1_traits["DOI"] = "http://teledetection.ipgp.jussieu.fr/opticleaf/lopex.htm"
ds1_traits = ds1_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds1_reflectance = ds1_reflectance.loc[:,"450":"2400"]
ds1_reflectance = do_resample(ds1_reflectance)
ds1 = pd.concat([ds1_reflectance, ds1_traits],axis = 1)
# ds1

### 2. Angers

In [4]:
ds2_reflectance = pd.read_csv('../1_datasets/0_original/2_Angers/angers2003reflectance.csv')
ds2_traits = pd.read_csv('../1_datasets/0_original/2_Angers/angers2003metadata.csv')
ds2_traits['Latitude'] = 47.4869517
ds2_traits['Longitude'] = -0.5333723

ds2_pft = ['Deciduous broadleaf forests','Shrublands','Deciduous broadleaf forests','Shrublands', 'Shrublands','Shrublands',
           'Deciduous broadleaf forests','Shrublands','Deciduous broadleaf forests','Shrublands','Deciduous broadleaf forests',
           'Shrublands','Shrublands','Shrublands','Vine','Vine','Deciduous broadleaf forests','Deciduous broadleaf forests',
           'Deciduous broadleaf forests','Shrublands','Shrublands','Deciduous broadleaf forests','Vine','Evergreen broadleaf forests',
           'Shrublands','Evergreen broadleaf forests','Deciduous broadleaf forests','Shrublands','Grasslands','Deciduous broadleaf forests',
           'Shrublands','Deciduous broadleaf forests','Shrublands','Shrublands','Evergreen broadleaf forests','Shrublands',
           'Shrublands','Shrublands','Shrublands']

for i in range(len(ds2_traits['English Name'].unique())):
    ds2_traits.loc[ds2_traits['English Name'] == ds2_traits['English Name'].unique()[i], 'PFT'] = ds2_pft[i]

ds2_traits.rename(columns={'Chlorophyll_a+b (µg/cm²)':'Chla+b','Carotenoid (µg/cm²)':'Ccar',
                           'Equivalent Water Thickness (g/cm²)':'EWT','Leaf mass per area (g/cm²)':'LMA'},inplace = True)
ds2_traits['Sample date'] = '2003'
ds2_traits['Dataset ID'] = 'Dataset#2'
ds2_traits['Site ID'] = 'Site#2'
ds2_traits['Dataset Name'] = 'Angers'
ds2_traits["Foreoptic type"] = np.nan
ds2_traits["Instruments"] = "ASD FieldSpec"
ds2_traits["Reference"] = "(Jacquemound et al., 2003)"
ds2_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/angers-leaf-optical-properties-database--2003-"
ds2_traits["DOI"] = "https://doi.org/10.1016/j.rse.2008.02.012"
ds2_traits = ds2_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds2_reflectance = ds2_reflectance.loc[:,"450":"2400"]
ds2_reflectance = do_resample(ds2_reflectance)
ds2 = pd.concat([ds2_reflectance, ds2_traits],axis = 1)
# ds2

### 3. Leaf trait and spectra seasonality—MV

In [5]:
ds3_traits = pd.read_csv('../1_datasets/0_original/3_Leaf trait and spectra seasonality_MV/2011_MV_leaftraits.csv')

ds3_traits['Latitude'] = 41.3616166
ds3_traits['Longitude'] = -70.5778501
ds3_traits.rename(columns={'Species':'English Name'},inplace = True)
ds3_traits['Latin Name'] = 'Quercus rubra'
ds3_traits['PFT'] = 'Deciduous broadleaf forests'
ds3_traits['LMA'] = ds3_traits['LMA']/10000

sample_date_MV = ['20110608','20110616','20110627','20110706','20110713','20110727','20110817','20110914','20110922','20110927','20111005','20111012','20111021','20111025','20111101','20111108']

for i in range(len(ds3_traits['DOY'].unique())):
    ds3_traits.loc[ds3_traits['DOY']== ds3_traits['DOY'].unique()[i], 'Sample date'] = sample_date_MV[i]

ds3_traits["Dataset ID"] = 'Dataset#3'
ds3_traits["Site ID"] = 'Site#3'

ds3_traits['Dataset Name'] = 'Leaf trait and spectra seasonality—MV'
ds3_traits["Foreoptic type"] = "Integrating sphere"
ds3_traits["Instruments"] = "ASD FieldSpec 4"
ds3_traits["Reference"] = "(Yang et al., 2016)"
ds3_traits["Link to original dataset"] = "https://github.com/geoxiyang/LeafTraitSpectraSeasonality-Yangetal2016RSE"
ds3_traits["DOI"] = "https://doi.org/10.1016/j.rse.2016.03.026"
ds3_traits = ds3_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
path = '../1_datasets/0_original/3_Leaf trait and spectra seasonality_MV/2011-MV/'
filename = os.listdir(path)
filename.sort()
reflectance = pd.read_csv(path + filename[0], header= None)
reflectance = reflectance.T
reflectance.columns = reflectance.iloc[0].astype(int).astype(str)
reflectance.drop(0,axis = 0, inplace = True)

for i in range(1,len(filename)):
    ref = pd.read_csv(path + filename[i], header= None)
    ref = ref.T
    ref.columns = ref.iloc[0].astype(int).astype(str)
    ref.drop(0,axis = 0, inplace = True)
    reflectance = pd.concat([reflectance, ref])
del reflectance['0']
ds3_reflectance = reflectance.copy()
ds3_reflectance.reset_index(drop = True, inplace = True)

ds3_reflectance = ds3_reflectance.loc[:,"450":"2400"]
ds3_reflectance = do_resample(ds3_reflectance)
ds3 = pd.concat([ds3_reflectance, ds3_traits],axis = 1)
# ds3

### 4. Leaf trait and spectra seasonality—HF

In [6]:
ds4_traits = pd.read_csv('../1_datasets/0_original/4_Leaf trait and spectra seasonality_HF/2012_HF_leaftraits.csv')
ds4_traits['Latitude'] = 42.538
ds4_traits['Longitude'] = -72.171
ds4_traits.rename(columns={'Species':'English Name'},inplace = True)
ds4_traits.loc[ds4_traits['English Name'] == 'Red Oak', 'Latin Name'] = 'Quercus rubra'
ds4_traits.loc[ds4_traits['English Name'] == 'Red Oak', 'PFT'] = 'Deciduous broadleaf forests'
ds4_traits.loc[ds4_traits['English Name'] == 'Red Maple', 'Latin Name'] = 'Acer rubrum'
ds4_traits.loc[ds4_traits['English Name'] == 'Red Maple', 'PFT'] = 'Deciduous broadleaf forests'
ds4_traits.loc[ds4_traits['English Name'] == 'Yellow Birch', 'Latin Name'] = 'Betula alleghaniensis'
ds4_traits.loc[ds4_traits['English Name'] == 'Yellow Birch', 'PFT'] = 'Deciduous broadleaf forests'
ds4_traits['LMA'] = ds4_traits['LMA']/10000

sample_date_HF = ['20120501','20120510','20120517','20120522','20120531','20120606','20120618','20120626','20120710','20120724','20120807','20120821','20120903','20120927','20121004','20121017']
for i in range(len(ds4_traits['DOY'].unique())):
    ds4_traits.loc[ds4_traits['DOY']== ds4_traits['DOY'].unique()[i], 'Sample date'] = sample_date_HF[i]

ds4_traits["Dataset ID"] = 'Dataset#4'
ds4_traits["Site ID"] = 'Site#4'
ds4_traits['Dataset Name'] = 'Leaf trait and spectra seasonality—HF'
ds4_traits["Foreoptic type"] = "Integrating sphere"
ds4_traits["Instruments"] = "ASD FieldSpec 4"
ds4_traits["Reference"] = "(Yang et al., 2016)"
ds4_traits["Link to original dataset"] = "https://github.com/geoxiyang/LeafTraitSpectraSeasonality-Yangetal2016RSE"
ds4_traits["DOI"] = "https://doi.org/10.1016/j.rse.2016.03.026"
ds4_traits = ds4_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

path = '../1_datasets/0_original/4_Leaf trait and spectra seasonality_HF/2012_HF/ref/'
filename = os.listdir(path)
filename.sort()
reflectance = pd.read_csv(path + filename[0], header= None)
reflectance = reflectance.T
reflectance.columns = reflectance.iloc[0].astype(int).astype(str)
reflectance.drop(0,axis = 0, inplace = True)

for i in range(1,len(filename)):
    ref = pd.read_csv(path + filename[i], header= None)
    ref = ref.T
    ref.columns = ref.iloc[0].astype(int).astype(str)
    ref.drop(0,axis = 0, inplace = True)
    reflectance = pd.concat([reflectance, ref])
del reflectance['0']
ds4_reflectance = reflectance.copy()
ds4_reflectance.reset_index(drop = True, inplace = True)

ds4_reflectance = ds4_reflectance.loc[:,"450":"2400"]
ds4_reflectance = do_resample(ds4_reflectance)
ds4 = pd.concat([ds4_reflectance, ds4_traits],axis = 1)
# ds4

### 5. 2018 Cedar Creek pressed leaves

In [7]:
df1_refl = pd.read_csv('../1_datasets/0_original/5_2018 Cedar Creek pressed leaves/pressed_spec_mn_rwc_avg.csv')
df1_traits = pd.read_csv('../1_datasets/0_original/5_2018 Cedar Creek pressed leaves/pressed_spec_mn_rwc_meta.csv')
df1_all = pd.merge(df1_refl,df1_traits, how = 'left', on = 'ID')

ds5_reflectance = df1_all.loc[:,'400':'2400']
ds5_traits = df1_all.loc[:,'Plot':]

ds5_traits.rename(columns={'functional.group':'PFT','full.species':'Latin Name'},inplace = True)
ds5_traits['Latitude'] = 45.4019246
ds5_traits['Longitude'] = -93.2003527

ds5_traits.loc[ds5_traits['PFT']=='conifer', 'PFT'] = 'Evergreen needleleaf forests'
ds5_traits.loc[ds5_traits['PFT']=='broadleaf', 'PFT'] = 'Deciduous broadleaf forests'
ds5_traits.loc[ds5_traits['PFT']=='herb', 'PFT'] = 'Grasslands'
ds5_species = ['Jack pine','Northern red oak','Northern pin oak','Red maple','Eastern white pine','White oak',
                'American basswood','Largeleaf wild indigo','Tall blazing star',"White four o'clock",'Bur oak',
                'Paper birch','Switchgrass','Eastern redcedar','Butterfly milkweed','Roundhead lespedeza',
                'Big bluestem','Stiff goldenrod','Red pine','Tall cinquefoil','Wild bergamot','Common milkweed',
                'Sundial lupine','Indiangrass','White poplar','Black willow','Gray goldenrod','quaking aspen']

for i in range(len(ds5_traits['Latin Name'].unique())):
    ds5_traits.loc[ds5_traits['Latin Name']== ds5_traits['Latin Name'].unique()[i], 'English Name'] = ds5_species[i]

ds5_traits['LMA'] = ds5_traits['LMA']/10
ds5_traits['EWT'] = ds5_traits['EWT']/10
ds5_traits['Sample date'] = '2018'
ds5_traits["Dataset ID"] = 'Dataset#5'
ds5_traits["Site ID"] = 'Site#5'
ds5_traits['Dataset Name'] = '2018 Cedar Creek pressed leaves'
ds5_traits["Foreoptic type"] = "Leaf clip"
ds5_traits["Instruments"] = "PSR 3500+"
ds5_traits["Reference"] = "(Kothari et al., 2022)"
ds5_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/2018-cedar-creek-pressed-leaves"
ds5_traits["DOI"] = "https://doi.org/10.21232/b5uXd859"
ds5_traits = ds5_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds5_reflectance = ds5_reflectance.loc[:,"450":"2400"]
ds5_reflectance = do_resample(ds5_reflectance)
ds5 = pd.concat([ds5_reflectance, ds5_traits],axis = 1)
# ds5

### 6. Fresh-leaf CABO spectra from herbarium project

In [8]:
ds6 = pd.read_csv('../1_datasets/0_original/6_Fresh-leaf CABO spectra from herbarium project/fresh_spec_avg_new.csv')

ds6_reflectance = ds6.loc[:,"450":]
ds6_traits = ds6.loc[:,:"Zn_area"]
ds6_traits = ds6_traits[['Species',"GrowthForm",'Latitude','Longitude','date_measured','LMA','EWT','chlA_area', 'chlB_area','car_area']]

ds6_traits['Latin Name'] = ds6_traits['Species']
ds6_traits.rename(columns={'date_measured':'Sample date'},inplace = True)
ds6_traits.loc[ds6_traits["GrowthForm"] == 'tree', 'PFT'] = 'Deciduous broadleaf forests'
ds6_traits.loc[ds6_traits["GrowthForm"] == 'herb', 'PFT'] = 'Grasslands'
ds6_traits.loc[ds6_traits["GrowthForm"] == 'shrub', 'PFT'] = 'Shrublands'
ds6_traits.loc[ds6_traits["GrowthForm"] == 'vine', 'PFT'] = 'Vine'

ds6_species = ['Gray birch','Red maple','Sugar Maple','Quaking aspen','Paper birch','Large-tooth aspen','American beech',
                'Northern red oak','Silver maple','Tall goldenrod','Common milkweed','Red raspberry','Redosier dogwood',
                'Riverbank grape','Canada goldenrod','Common reed','Sandbar willow','Flat-top goldentop','Spreading dogbane',
                'Reed canarygrass','Bluejoint','Royal fern','Interrupted fern','White meadowsweet','Sweetgale','Alternateleaf dogwood',
                'American black elderberry','Smooth rose','Nannyberry','Purpleflowering raspberry','Woodland sunflower','Sitka alder',
                'American cranberrybush','White meadowsweet','Willow','Mountain fly honeysuckle','Russian olive','American elm','Green ash',
                'Eastern cottonwood','Sensitive fern','Boxelder','Common reed','Jerusalem artichoke','American bladdernut','Obedient plant',
                'Pickerelweed','Bog Labrador tea','Leatherleaf','Catberry','Balsam poplar','Canada anemone','Hobblebush','Beaked hazelnut',
                'Pin cherry','Striped maple','Mountain maple','Black cherry','White ash','Littleleaf linden','Bitternut hickory','Hophornbeam',
                'Gray alder','Butternut','Yellow birch','Agonis flexuosa']

for i in range(len(ds6_traits['Latin Name'].unique())):
    ds6_traits.loc[ds6_traits['Latin Name']== ds6_traits['Latin Name'].unique()[i], 'English Name'] = ds6_species[i]

ds6_traits['LMA'] = ds6_traits['LMA']/10
ds6_traits['EWT'] = ds6_traits['EWT']/10
ds6_traits['Ccar'] = ds6_traits['car_area']*10**6
ds6_traits['Chla'] = ds6_traits['chlA_area']*10**6
ds6_traits['Chlb'] = ds6_traits['chlB_area']*10**6
ds6_traits['Chla+b'] = ds6_traits['Chla']+ds6_traits['Chlb']


ds6_traits["coord"] = [xy for xy in zip(round(ds6_traits["Latitude"],1), round(ds6_traits["Longitude"],1))]
sites = {(45.5, -73.3):"Site#6", (45.6, -73.3):"Site#7", (45.4, -75.5):"Site#8", (46.0, -74.0):"Site#9",
         (45.5, -74.0):"Site#10", (45.5, -74.1):"Site#11", (45.6, -73.4):"Site#12", (45.6, -73.5):"Site#13",
         (45.6, -73.6):"Site#14", (-34.6, 115.9):"Site#15", (-34.8, 116.1):"Site#16"}

ds6_traits["Site ID"] = [sites[x] for x in ds6_traits["coord"]]
ds6_traits["Dataset ID"] = 'Dataset#6'
ds6_traits['Dataset Name'] = 'Fresh-leaf CABO spectra from herbarium project'
ds6_traits["Foreoptic type"] = "Integrating sphere"
ds6_traits["Instruments"] = "SVC HR-1024i"
ds6_traits["Reference"] = "(Kothari et al., 2022)"
ds6_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-cabo-spectra-from-herbarium-project"
ds6_traits["DOI"] = "https://doi.org/10.21232/deP7jVyq"

ds6_traits = ds6_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
ds6_reflectance = ds6_reflectance.loc[:,"450":"2400"]
ds6_reflectance = do_resample(ds6_reflectance)
ds6 = pd.concat([ds6_reflectance, ds6_traits],axis = 1)
# ds6

### 7. Fresh Leaf Spectra to Estimate Foliar Functional Traits across NEON domains

In [9]:
df3_all = pd.read_csv('../1_datasets/0_original/7_Fresh Leaf Spectra to Estimate Foliar Functional Traits across NEON domains/pigments_phenolics_all.csv')
ds7_reflectance = df3_all.loc[:,'350':'2500']
ds7_traits = df3_all.loc[:,'Sample_ID':'Phenolics(mg/g)']

ds7_traits['Latitude'] = 0
ds7_traits['Longitude'] = 0

for i in ds7_traits['Site_Name'].unique():
    geolocator = Nominatim(user_agent='BuyiXiao',timeout=5)
    location = geolocator.geocode(i)
    if location is None:
        pass
    else:
        ds7_traits.loc[ds7_traits['Site_Name'] == i, 'Latitude'] = location.latitude
        ds7_traits.loc[ds7_traits['Site_Name'] == i, 'Longitude'] = location.longitude

ds7_traits.loc[ds7_traits['Site_Name'] == 'McDiffett Creek', 'Latitude'] = 38.9447
ds7_traits.loc[ds7_traits['Site_Name'] == 'McDiffett Creek', 'Longitude'] = -96.4421

ds7_traits.loc[ds7_traits['Site_Name'] == 'Konza Prairie Biological Station', 'Latitude'] = 39.0546
ds7_traits.loc[ds7_traits['Site_Name'] == 'Konza Prairie Biological Station', 'Longitude'] = -96.7307

ds7_traits.loc[ds7_traits['Site_Name'] == 'University of Kansas Field Station', 'Latitude'] = 38.9822
ds7_traits.loc[ds7_traits['Site_Name'] == 'University of Kansas Field Station', 'Longitude'] = -95.2877

ds7_traits.loc[ds7_traits['Site_Name'] == 'Northern Great Plains Research Laboratory', 'Latitude'] = 46.7697
ds7_traits.loc[ds7_traits['Site_Name'] == 'Northern Great Plains Research Laboratory', 'Longitude'] = -100.9154

ds7_traits.loc[ds7_traits['Site_Name'] == 'Steigerwaldt-Chequamegon', 'Latitude'] = 45.50894
ds7_traits.loc[ds7_traits['Site_Name'] == 'Steigerwaldt-Chequamegon', 'Longitude'] = -89.58637

ds7_traits.loc[ds7_traits['Site_Name'] == 'The Jones Center At Ichauway', 'Latitude'] = 31.1948
ds7_traits.loc[ds7_traits['Site_Name'] == 'The Jones Center At Ichauway', 'Longitude'] = -84.4686

ds7_traits.loc[ds7_traits['Site_Name'] == 'Harvard Forest & Quabbin Watershed', 'Latitude'] = 42.53691
ds7_traits.loc[ds7_traits['Site_Name'] == 'Harvard Forest & Quabbin Watershed', 'Longitude'] = -72.17265

ds7_traits.loc[ds7_traits['Site_Name'] == 'Lenoir Landing', 'Latitude'] = 32.1237024
ds7_traits.loc[ds7_traits['Site_Name'] == 'Lenoir Landing', 'Longitude'] = -88.2464611

ds7_latin_name = ['Quercus coccinea Münchh.', 'Madia madioides(Nutt.) Greene', 'Salix nigra', 'Macromeria viridiflora', 'Oxydendrum arboreum(L.) DC.', 'Conyza floribunda', 'Quercus stellata', 'Nymphaea sagittifolia', 'Calla palustris', 'Quercus nigra', 'Albizia julibrissin', 'Quercus lyrata', 'Carex illota', 'Quercus acuminata(Michx.) Houba', 'Quercus ×palaeolithicola', 'Celtis laevigata', 'Nymphoides aquatica (J.F. Gmel.) Kuntze', 'Juglans cinerea', 'Calliandra conferta', 'Hackelia diffusa(Lehm.) I.M. Johnst.', 'Ulmus americana', 'Distichlis spicata (L.) Greene', 'Rhus glabra', 'Gleditsia triacanthos', 'Syringa vulgaris', 'Ailanthus altissima(Mill.) Swingle', 'Elaeagnus angustifolia', np.nan, 'Quercus velutina', 'Vitis riparia', 'Asimina triloba(L.) Dunal', 'Campanula glomerata', 'Magnolia acuminata(L.) L.', 'Betula lenta', 'Proserpinaca pectinata', 'Quercus mohriana', 'Palafoxia rosea(Bush) Cory', 'Calochortus tolmiei', 'Linum striatum', 'Quercus falcata', 'cer saccharum', 'Poa gracillima', 'Quercus macrocarpa', 'Quercus laceyi', 'Quercus phellos', 'Quercus shumardii', 'Melia azedarach', 'Acer floridanum(Chapm.) Pax', 'Quercus virginiana Mill.', 'Sabal palmetto(Walter) Lodd. ex Schult. & Schult. f.', 'Gordonia lasianthus(L.) Ellis', 'Spartina pectinata', 'Koeleria macrantha(Ledeb.) Schult.', 'Andropogon gerardii Vitman', 'Trichomanes davallioides', 'Panicum virgatum', 'Sophora nuttalliana', 'Schizachyrium scoparium(Michx.) Nash', 'Phacelia argentea', 'Typha latifolia', 'Phacelia austromontana', 'Amaranthus arenicola', 'Teucrium laciniatum', 'Amaranthus californicus(Moq.) S. Watson', 'Apocynum cannabinum', 'Ambrosia trifida', 'Sicyos laciniatus', 'Rhus copallinum', 'Cornus drummondii', 'Shepherdia argentea(Pursh) Nutt.', 'Rhamnus californica', 'Maranthes corymbosa', 'Rubus allegheniensis', 'Allionia incarnata', 'Sedum reflexum', 'Quercus michauxii', 'Pinus taeda', 'Tabernaemontana divaricata(L.) R.', 'Juniperus virginiana', 'Pinus sylvestris', 'Picea pungens', 'Pinus strobus', 'Tsuga canadensis(L.) Carrière', 'Pinus rigida', 'Thalictrum occidentale', 'Larix laricina(Du Roi) K. Koch', 'Abies balsamea(L.) Mill.', 'Picea mariana(Mill.) Britton, Sterns & Poggenb.', 'Pinus balfouriana', 'Pinus resinosa', 'Picea glauca(Moench) Voss', 'Pinus glabra', 'Picris echioides', 'Pinaropappus parvus', 'Talinum calycinum', 'Picea abies(L.) Karst.', 'Pinus nigra', 'Abutilon berlandieri', 'Juncus nevadensis', 'Lagenaria siceraria', 'Pilea nummulariifolia', 'Labordia kaalae', 'Pinus koraiensis', 'Pseudotsuga menziesii', 'Pilea nummulariifolia', 'Picramnia pentandra', 'Acacia saligna(Labill.) ', 'Acer rubrum', 'Carya ovalis', 'Fraxinus americana', 'Quercus alba', 'Solanum campechiense', 'Quercus rubra', 'Betula papyrifera', 'Machaeranthera grindelioides(Nutt.) Shinners', 'Quercus ×heterophylla', 'Polycarpon depressum', 'Fraxinus pennsylvanica', 'Monarda russeliana', 'Arnica snyderi', 'Quercus ×macnabiana', 'Liriodendron tulipifera', 'Platanus occidentalis', 'Poa tracyi', 'Populus balsamifera', 'Alnus rubra', 'Acer macrophyllum', 'Acer circinatum', 'Arctostaphylos virgata', 'Bromus diandrus', 'Quercus douglasii', 'Quercus willdenowiana(Dippel) ', 'Aesculus californica(Spach) Nutt.', 'Brodiaea howellii', 'Oldenlandia strigulosa', 'Sagittaria lancifolia', 'Quercus chapmanii', 'Cocos nucifera', 'Allium rhizomatum', 'Quercus kelloggii', 'Acacia schottii', 'Acacia smallii', 'Fagus grandifolia', 'Calochortus greenei', 'Pascopyrum smithii', 'Medicago sativa L.', 'Polygala barbeyana']
ds7_english_name = ['Scarlet oak', 'Woodland madia', 'Black willow', 'Giant-trumpets', 'Sourwood', 'Asthmaweed', 'Post oak', 'Yellow pond-lily', 'Water arum', 'Water oak', 'Silktree', 'Overcup oak', 'Sheep sedge', 'Mapleleaf oak', 'Hybrid oak', 'Sugarberry', 'Big floatingheart', 'Butternut', 'Rio Grande stickpea', 'Spreading stickseed', 'American elm', 'Saltgrass', 'Smooth sumac', 'Honeylocust', 'Common lilac', 'Tree of heaven', 'Russian olive', np.nan, 'Black oak', 'Riverbank grape', 'Pawpaw', "Dane's blood", 'Cucumber tree', 'Sweet birch', 'Combleaf mermaidweed', 'Mohr oak', 'Rosy palafox', 'Tolmie star-tulip', 'Ridged yellow flax', 'Southern red oak', 'Black maple', 'Slender bluegrass', 'Bur oak', 'Lacey oak', 'Willow oak', "Shumard's oak", 'Chinaberrytree', 'Southern sugar maple', 'Live oak', 'Cabbage palmetto', 'Loblolly bay', 'Prairie cordgrass', 'Prairie Junegrass', 'Big bluestem', 'Kilau', 'Switchgrass', 'Silky sophora', 'Little bluestem', 'Sanddune phacelia', 'Broadleaf cattail', 'Southern Sierra phacelia', 'Sandhill amaranth', 'Lacy germander', 'California amaranth', 'Indianhemp', 'Great ragweed', 'Cutleaf bur cucumber', 'Winged sumac', 'Roughleaf dogwood', 'Silver buffaloberry', 'California buckthorn', 'Bkau', 'Allegheny blackberry', 'Trailing windmills', "Jenny's stonecrop", 'Swamp chestnut oak', 'Loblolly pine', 'Pinwheelflower', 'Eastern redcedar', 'Scots pine', 'Blue spruce', 'Eastern white pine', 'Eastern hemlock', 'Pitch pine', 'Western meadow-rue', 'Tamarack', 'Balsam fir', 'Black spruce', 'Foxtail pine', 'Red pine', 'White spruce', 'Spruce pine', 'Bristly oxtongue', 'Small rocklettuce', 'Largeflower fameflower', 'Norway spruce', 'Austrian pine', "Berlandier's Indian mallow", 'Sierra rush', 'Bottle gourd', 'Creeping charlie', 'Waianae Range labordia', 'Chinese pinenut', 'Douglas-fir', 'Creeping charlie', 'Florida bitterbush', 'Orange wattle', 'Red maple', 'Red hickory', 'White ash', 'White oak', 'Redberry nightshade', 'Northern red oak', 'Paper birch', 'Rayless tansyaster', 'Bartram oak', 'California manyseed', 'Green ash', 'Redpurple beebalm', 'Snow arnica', 'MacNab oak', 'Tuliptree', 'American sycamore', "Tracy's bluegrass", 'Balsam poplar', 'Red alder', 'Bigleaf maple', 'Vine maple', 'Bolinas manzanita', 'Great brome', 'Blue oak', 'Hybrid oak', 'California buckeye', 'Soft brome', 'Rough oldenlandia', 'Bulltongue arrowhead', 'Chapman oak', 'Coconut palm', 'Red flower onion', 'California black oak', "Schott's acacia", 'Sweet acacia', 'American beech', "Greene's mariposa lily", 'Western wheatgrass', 'Alfalfa', 'Blue milkwort']
ds7_pft = ['Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', np.nan, 'Deciduous broadleaf forests', 'Vine', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Shrublands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Evergreen broadleaf forests', 'Shrublands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Deciduous needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Grasslands', 'Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Shrublands', 'Grasslands', 'Vine', 'Grasslands', 'Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands']

for i in range(len(ds7_traits['Species'].unique())):
    ds7_traits.loc[ds7_traits['Species']== ds7_traits['Species'].unique()[i], 'Latin Name'] = ds7_latin_name[i]
    ds7_traits.loc[ds7_traits['Species']== ds7_traits['Species'].unique()[i], 'English Name'] = ds7_english_name[i]
    ds7_traits.loc[ds7_traits['Species']== ds7_traits['Species'].unique()[i], 'PFT'] = ds7_pft[i]

ds7_traits.rename(columns={'Chl_AB_area(ug/cm2)':'Chla+b','Car_area(ug/cm2)':'Ccar','Sample_Date':'Sample date'},inplace = True)

k = 17
for i in ds7_traits['Site_Name'].unique():
    ds7_traits.loc[ds7_traits['Site_Name'] ==i,'Site ID'] = 'Site#'+str(k)  
    k = k+1

ds7_traits["Dataset ID"] = 'Dataset#7'
ds7_traits['Dataset Name'] = 'Fresh Leaf Spectra to Estimate Foliar Functional Traits across NEON domains'
ds7_traits["Foreoptic type"] = "Contact probe"
ds7_traits["Instruments"] = "PSR 3500+"
ds7_traits["Reference"] = "(Wang, 2022)"
ds7_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-spectra-to-estimate-foliar-functional-traits-across-neon-domains"
ds7_traits["DOI"] = "https://doi.org/10.21232/EYp6nn9Z"

ds7_traits = ds7_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds7_reflectance = ds7_reflectance.loc[:,"450":"2400"]
ds7_reflectance = do_resample(ds7_reflectance)
ds7 = pd.concat([ds7_reflectance, ds7_traits],axis = 1)
# ds7

### 8. Seasonal fresh leaf spectra and traits, Blackhawk Island-WI

In [10]:
ds8_reflectance = pd.read_csv('../1_datasets/0_original/8_Seasonal fresh leaf spectra and traits, Blackhawk Island-WI/bhi2018_fresh_spectra.csv')
ds8_reflectance = ds8_reflectance.loc[:,'350':]
ds8_reflectance = ds8_reflectance/100.0

ds8_traits = pd.read_csv('../1_datasets/0_original/8_Seasonal fresh leaf spectra and traits, Blackhawk Island-WI/bhi2018_fresh_data.csv')
ds8_traits.rename(columns={'species': 'Latin Name', 'common_name': 'English Name','veg_type':"PFT",'latitude':'Latitude', 'longitude':'Longitude',
                           'lma_g_m2':'LMA','sample_date':'Sample date'},inplace = True)

ds8_traits['PFT'] = 'Deciduous broadleaf forests'

ds8_traits['LMA'] = ds8_traits['LMA']/10000
ds8_traits["Dataset ID"] = 'Dataset#8'
ds8_traits['Site ID'] = 'Site#42'

ds8_traits['Dataset Name'] = 'Seasonal fresh leaf spectra and traits, Blackhawk Island-WI'
ds8_traits["Foreoptic type"] = "Leaf clip"
ds8_traits["Instruments"] = "PSR 3500+"
ds8_traits["Reference"] = "(Chlus & Townsend, 2022)"
ds8_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/seasonal-fresh-leaf-spectra-and-traits--blackhawk-island--wi"
ds8_traits["DOI"] = "https://doi.org/10.1016/j.rse.2022.113023"
ds8_traits = ds8_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
# ds8_reflectance
ds8_traits = ds8_traits[ds8_traits["LMA"]>0]
ds8_reflectance = ds8_reflectance.iloc[ds8_traits.index]

ds8_reflectance.reset_index(drop = True, inplace = True)
ds8_traits.reset_index(drop = True, inplace = True)

ds8_reflectance = ds8_reflectance.loc[:,"450":"2400"]
ds8_reflectance = do_resample(ds8_reflectance)
ds8 = pd.concat([ds8_reflectance, ds8_traits],axis = 1)
# ds8

### 9. Leaf reflectance and traits of floating and emergent macrophytes

In [11]:
ds9_reflectance = pd.read_csv('../1_datasets/0_original/9_Leaf reflectance and traits of floating and emergent macrophytes/leaf_spectra.csv')
ds9_reflectance = ds9_reflectance.loc[:,'345':]

ds9_traits = pd.read_csv('../1_datasets/0_original/9_Leaf reflectance and traits of floating and emergent macrophytes/leaf_traits.csv')

ds9_traits.loc[ds9_traits['Lake1'] == 'Kis-Balaton', 'Latitude'] = 46.631317
ds9_traits.loc[ds9_traits['Lake1'] == 'Kis-Balaton', 'Longitude'] = 17.12785

ds9_traits.loc[ds9_traits['Lake1'] == 'Mantova', 'Latitude'] = 45.1651953
ds9_traits.loc[ds9_traits['Lake1'] == 'Mantova', 'Longitude'] = 10.7519177

ds9_traits.loc[ds9_traits['Lake1'] == 'Varese', 'Latitude'] = 45.8252503
ds9_traits.loc[ds9_traits['Lake1'] == 'Varese', 'Longitude'] = 8.7378585

ds9_traits.rename(columns={'USDAsymbol': 'usda_code','CaS':'Chla','CbS': 'Chlb','CxcS':'Ccar','year':'Sample date'},inplace = True)

ds9_latin_name = ['Nelumbo nucifera Gaertn.','Nuphar lutea (L.) Sm.','Phragmites australis',
                   'Trapa natans L.','Nymphaea alba L.','Ludwigia hexapetala']
ds9_english_name = ['Sacred lotus','Yellow pond-lily','European common reed','Water chestnut',
                     'European white waterlily','Water primrose']

for i in range(len(ds9_traits['usda_code'].unique())):
    ds9_traits.loc[ds9_traits['usda_code']== ds9_traits['usda_code'].unique()[i], 'Latin Name'] = ds9_latin_name[i]
    ds9_traits.loc[ds9_traits['usda_code']== ds9_traits['usda_code'].unique()[i], 'English Name'] = ds9_english_name[i]

ds9_traits['PFT'] = 'Grasslands'
ds9_traits['Chla+b'] = ds9_traits['Chla']+ds9_traits['Chlb']
ds9_traits['LMA'] = ds9_traits['LMA']/10000
ds9_traits["Dataset ID"] = 'Dataset#9'
ds9_traits['Dataset Name'] = 'Leaf reflectance and traits of floating and emergent macrophytes'
ds9_traits["Foreoptic type"] = "Contact probe"
ds9_traits["Instruments"] = "PSR 3500+"
ds9_traits["Reference"] = "(Villa et al., 2021)"
ds9_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-reflectance-and-tratis-of-floating-and-emergent-macrophytes"
ds9_traits["DOI"] = "https://doi.org/10.21232/uq4HsfSY" 
k = 43
for i in ds9_traits['Lake1'].unique():
    ds9_traits.loc[ds9_traits['Lake1'] ==i,'Site ID'] = 'Site#'+str(k)  
    k = k+1
    
ds9_traits = ds9_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds9_traits = ds9_traits[(ds9_traits["Chla+b"]>0)|(ds9_traits["Ccar"]>0)|(ds9_traits["LMA"]>0)]
ds9_reflectance = ds9_reflectance.iloc[ds9_traits.index]
ds9_reflectance.reset_index(drop = True, inplace = True)
ds9_traits.reset_index(drop = True, inplace = True)
ds9_reflectance = ds9_reflectance.loc[:,"450":"2401"]
ds9_reflectance = do_resample(ds9_reflectance)
ds9 = pd.concat([ds9_reflectance, ds9_traits],axis = 1)
# ds9

### 10. Seasonal measurements of photosynthesis and leaf traits in scarlet oak

In [12]:
ds10_reflectance = pd.read_csv('../1_datasets/0_original/10_Seasonal measurements of photosynthesis and leaf traits in scarlet oak/leaf_spectra.csv')
ds10_reflectance = ds10_reflectance.loc[:,'350':]
ds10_traits = pd.read_csv('../1_datasets/0_original/10_Seasonal measurements of photosynthesis and leaf traits in scarlet oak/leaf_traits_and_photosynthesis.csv')

ds10_traits['Latitude'] = 40.864466
ds10_traits['Longitude'] = -72.875158
ds10_traits.rename(columns={'Species': 'Latin Name','Measurement_Date':'Sample date'},inplace = True)
ds10_traits['English Name'] = 'Scarlet oak'
ds10_traits['PFT'] = 'Deciduous broadleaf forests'
ds10_traits['LMA'] = ds10_traits['LMA']/10000

ds10_traits["Dataset ID"] = 'Dataset#10'
ds10_traits['Site ID'] = 'Site#46'
ds10_traits['Dataset Name'] = 'Seasonal measurements of photosynthesis and leaf traits in scarlet oak'
ds10_traits["Foreoptic type"] = "Leaf clip"
ds10_traits["Instruments"] = "SVC LC-RP-Pro"
ds10_traits["Reference"] = "(Burnett et al., 2021)"
ds10_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/seasonal-measurements-of-photosynthesis-and-leaf-traits-in-scarlet-oak"
ds10_traits["DOI"] = "https://doi.org/10.21232/ujBYNxhm" 
ds10_traits = ds10_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds10_reflectance = ds10_reflectance.loc[:,"450":"2400"]
ds10_reflectance = do_resample(ds10_reflectance)
ds10 = pd.concat([ds10_reflectance, ds10_traits],axis = 1)
# ds10

### 11. Leaf and canopy spectroscopy and biochemical data of field-grown Cucurbita pepo under two stresses

In [13]:
df4_reflectance = pd.read_csv('../1_datasets/0_original/11_Leaf and canopy spectroscopy and biochemical data of field-grown Cucurbita pepo under two stresses/leaf_spectra.csv')
df4_meta = pd.read_csv('../1_datasets/0_original/11_Leaf and canopy spectroscopy and biochemical data of field-grown Cucurbita pepo under two stresses/metadata.csv')
df4_traits = pd.read_csv('../1_datasets/0_original/11_Leaf and canopy spectroscopy and biochemical data of field-grown Cucurbita pepo under two stresses/leaf_traits.csv')

df4_all = pd.merge(df4_reflectance, df4_meta, how = 'left', on = 'uniquefield')
df4_all = pd.merge(df4_all, df4_traits, how = 'left', on = 'uniquefield')

ds11_reflectance = df4_all.loc[:,'350':'2500']
ds11_traits = df4_all.loc[:,'Location':]
ds11_traits['Species'] = 'Cucurbita pepo'
ds11_traits['Latitude'] = 40.864466
ds11_traits['Longitude'] = -72.875158

ds11_traits.rename(columns={'Species': 'Latin Name','Chlorophyll':'Chla+b','Measurement_Date':'Sample date'},inplace = True)
ds11_traits['English Name'] ='Field Pumpkin'
ds11_traits['PFT'] = 'Grasslands'
ds11_traits['Chla+b'] = 6.34299*np.exp(ds11_traits['Chla+b']*0.043)-6.10629
ds11_traits['LMA'] = ds11_traits['LMA']/10000

sample_date = ['20190703','20190709','20190715','20190716',np.nan,'20190719','20190724','20190726','20190730','20190802','20190806']
for i in range(len(ds11_traits['Sample date'].unique())):
    ds11_traits.loc[ds11_traits['Sample date'] == ds11_traits['Sample date'].unique()[i], 'Sample date'] = sample_date[i]

ds11_traits["Dataset ID"] = 'Dataset#11'
ds11_traits["Site ID"] = 'Site#46'
ds11_traits['Dataset Name'] = 'Leaf and canopy spectroscopy and biochemical data of field-grown Cucurbita pepo under two stresses'
ds11_traits["Foreoptic type"] = "Leaf clip"
ds11_traits["Instruments"] = "PSR+"
ds11_traits["Reference"] = "(Burnett et al., 2021)"
ds11_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-and-canopy-spectroscopy-and-biochemical-data-of-field-grown-cucurbita-pepo-under-two-stresses"
ds11_traits["DOI"] = "https://doi.org/10.21232/RLmYbmE3"
ds11_traits = ds11_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds11_traits = ds11_traits[(ds11_traits["Chla+b"]>0)|(ds11_traits["LMA"]>0)]
ds11_reflectance = ds11_reflectance.iloc[ds11_traits.index]
ds11_reflectance.reset_index(drop = True, inplace = True)
ds11_traits.reset_index(drop = True, inplace = True)

ds11_reflectance = ds11_reflectance.loc[:,"450":"2400"]
ds11_reflectance = do_resample(ds11_reflectance)
ds11 = pd.concat([ds11_reflectance, ds11_traits],axis = 1)
# ds11

### 12. NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017

In [14]:
df5_reflectance = pd.read_csv('../1_datasets/0_original/12_NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017/ngee-tropics_puerto_rico_march2017_leaf_spectral_reflectance.csv')

df5_traits1 = pd.read_csv('../1_datasets/0_original/12_NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017/ngee-tropics_puerto_rico_march2017_lma.csv')
df5_traits1 = df5_traits1.iloc[:,0:4]
df5_traits2= pd.read_csv('../1_datasets/0_original/12_NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017/puertorico2017_cn.csv')

df5_meta = pd.read_csv('../1_datasets/0_original/12_NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017/ngee-tropics_puerto_rico_march2017_leaf_sample_detailv2.csv')

df5_all = pd.merge(df5_reflectance, df5_meta, how = 'left', on = 'SampleID')
df5_all = pd.merge(df5_all, df5_traits1, how = 'left', on = 'SampleID')
df5_all = pd.merge(df5_all, df5_traits2, how = 'left', on = 'SampleID')

ds12_reflectance = df5_all.loc[:,'350':'2500']
ds12_reflectance = ds12_reflectance/100.0

ds12_traits = df5_all.loc[:,'Site':]
ds12_traits.loc[ds12_traits['Site'] == 'FLMM+IITF ' , 'Site'] = 'FLMM+IITF'
ds12_traits.rename(columns={'Full_Species_Name':'Latin Name','Species': 'English Name','Date':'Sample date'},inplace = True)

ds12_pft = ['Shrublands','Evergreen broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Shrublands','Evergreen broadleaf forests','Shrublands','Shrublands','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests']

for i in range(len(ds12_traits['Latin Name'].unique())):
    ds12_traits.loc[ds12_traits['Latin Name'] == ds12_traits['Latin Name'].unique()[i], 'PFT'] = ds12_pft[i]

ds12_traits['LMA'] = ds12_traits['LMA']/10000


ds12_traits["Dataset ID"] = 'Dataset#12'
ds12_traits['Dataset Name'] = 'NGEE Tropics GLiHT Puerto Rico Campaign Leaf Spectral Reflectance and Transmittance March 2017'
ds12_traits["Foreoptic type"] = "Leaf clip, Integrating sphere"
ds12_traits["Instruments"] = "SVC HR-1024i, PSR Plus"
ds12_traits["Reference"] = "(Serbin et al., 2019)"
ds12_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-tropics-gliht-puerto-rico-campaign-leaf-spectral-reflectance-and-transmittance-march-2017"
ds12_traits["DOI"] = "https://doi.org/http://dx.doi.org/10.15486/ngt/1495204"
k = 47
s = ['FLMM', 'CAM II', 'CAR I', 'IITF', 'SB2']
for i in s:
    ds12_traits.loc[ds12_traits['Site'] ==i,"Site ID"] = 'Site#'+str(k)  
    k = k+1
ds12_traits.loc[ds12_traits['Site']=='FLMM+IITF',"Site ID"]= 'Site#50'

ds12_traits = ds12_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds12_traits = ds12_traits[ds12_traits["LMA"]>0]
ds12_reflectance = ds12_reflectance.iloc[ds12_traits.index]
ds12_reflectance.reset_index(drop = True, inplace = True)
ds12_traits.reset_index(drop = True, inplace = True)

ds12_reflectance = ds12_reflectance.loc[:,"450":"2400"]
ds12_reflectance = do_resample(ds12_reflectance)
ds12 = pd.concat([ds12_reflectance, ds12_traits],axis = 1)
# ds12

### 13. NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane

In [15]:
df6_reflectance = pd.read_csv('../1_datasets/0_original/13_NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane/2017_leaf_spectra_byagev2.csv')
df6_reflectance_meta = pd.read_csv('../1_datasets/0_original/13_NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane/2017_leaf_spectra_byagev2_metadata.csv')
df6_traits1 = pd.read_csv('../1_datasets/0_original/13_NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane/2017_slz_cn_byagev2_data.csv')
df6_traits2 = pd.read_csv('../1_datasets/0_original/13_NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane/2017_slz_aci_vcmax_data.csv')

df6_refl_merge = pd.merge(df6_reflectance,df6_reflectance_meta,how = 'left', on = 'SampleID')

df6_all = pd.merge(df6_refl_merge,df6_traits1,how = 'left', on = 'SampleID')
df6_all = pd.merge(df6_all,df6_traits2,how = 'left', on = 'SampleID')
df6_all = df6_all[(df6_all['LMA'].notnull())|(df6_all['Vcmax'].notnull())]

ds13_reflectance = df6_all.loc[:,'350':'2500']
ds13_reflectance = ds13_reflectance/100.0

ds13_traits = df6_all.loc[:,'Site':]
ds13_reflectance.reset_index(drop = True, inplace = True)
ds13_traits.reset_index(drop = True, inplace = True)

ds13_traits['Latitude'] = 18.2704541
ds13_traits['Longitude'] = -66.2051346

ds13_traits.rename(columns={'Genus_species': 'Latin Name','Date':'Sample date'},inplace = True)
ds13_pft = ['Evergreen broadleaf forests', 'Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests',np.nan]
ds13_english_name = ['Quaruba', 'Johnnyberry','White olive', 'Guatteria lucens', np.nan]
for i in range(len(ds13_traits['Latin Name'].unique())):
    ds13_traits.loc[ds13_traits['Latin Name'] == ds13_traits['Latin Name'].unique()[i], 'English Name'] = ds13_english_name[i]
    ds13_traits.loc[ds13_traits['Latin Name'] == ds13_traits['Latin Name'].unique()[i], 'PFT'] = ds13_pft[i]

ds13_traits['LMA'] = ds13_traits['LMA']/10000
ds13_traits["Dataset ID"] = 'Dataset#13'
ds13_traits["Site ID"] = 'Site#52'
ds13_traits['Dataset Name'] = 'NGEE Tropics February 2017 Leaf Spectral Reflectance Measured in Panama at the PA-SLZ Canopy Crane'
ds13_traits["Foreoptic type"] = "Leaf clip"
ds13_traits["Instruments"] = "SVC HR-2014i"
ds13_traits["Reference"] = "(Serbin et al., 2021)"
ds13_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-tropics-february-2017-leaf-spectral-reflectance-measured-in-panama-at-the-pa-slz-canopy-crane"
ds13_traits["DOI"] = "https://doi.org/http://dx.doi.org/10.15486/ngt/1475180"

ds13_traits = ds13_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
ds13_traits = ds13_traits[ds13_traits["LMA"]>0]
ds13_reflectance = ds13_reflectance.iloc[ds13_traits.index]
ds13_reflectance.reset_index(drop = True, inplace = True)
ds13_traits.reset_index(drop = True, inplace = True)

ds13_reflectance = ds13_reflectance.loc[:,"450":"2400"]
ds13_reflectance = do_resample(ds13_reflectance)
ds13 = pd.concat([ds13_reflectance, ds13_traits],axis = 1)
# ds13

### 14. NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016

In [16]:
df7_refl_meta = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/ngee-tropics_panama_2016_se_svc_feb_mar_apr_leaf_spectra_metadata.csv')
df7_refl_svc = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/ngee-tropics_panama_2016_svc_feb_mar_apr_leaf_spectra.csv')
df7_refl_se = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/ngee-tropics_panama_2016_se_mar_spectra_scaled.csv')

df7_sample_meta = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/2016enso_panama_leafspectrasamples_v2.csv')
df7_traits1 = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/2016enso_panama_lma1_v2.csv')
df7_traits2 = pd.read_csv('../1_datasets/0_original/14_NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016/2016enso_panama_chn_v2.csv')

df7_refl = pd.concat([df7_refl_svc,df7_refl_se])
df7_refl.reset_index(drop = True, inplace = True)
df7_refl_merge = pd.merge(df7_refl,df7_refl_meta,how = 'left',on = 'Sample_ID')

df7_all = pd.merge(df7_refl_merge,df7_sample_meta,how = 'left',on = 'Sample_ID')
df7_all = pd.merge(df7_all,df7_traits1,how = 'left',on = 'Sample_ID')
df7_all = pd.merge(df7_all,df7_traits2,how = 'left',on = 'Sample_ID')

ds14_reflectance = df7_all.loc[:,'350':'2500']
ds14_reflectance = ds14_reflectance/100.0

ds14_traits = df7_all.loc[:,'Site':]
ds14_reflectance.reset_index(drop = True, inplace = True)
ds14_traits.reset_index(drop = True, inplace = True)


ds14_traits['Latitude'] = 8.9936714
ds14_traits['Longitude']= -79.5463903

ds14_traits.rename(columns={'Genus_species': 'Latin Name','LMA_gDW_m2':'LMA','Sample_Date':'Sample date'},inplace = True)

ds14_latin_name = ['Albizia adinocephala','Calycophyllum candidissimum', 'Pittoniotis trichantha','Castilla elastica','Ficus insipida','Luehea seemannii','Cordia alliodora','Anacardium excelsum','Cecropia peltata','Guatteria dumetorum','Terminalia amazonia','Tocoyena pittieri','Tachigali versicolor','Carapa guianensis','Vochysia ferruginea','Apeiba membranacea', np.nan,'Miconia borealis','Virola multiflora','Callichlamys latifolia', 'Astronium graveolens','Posoqueria latifolia','Capparis indica','Piper reticulatum','Cojoba rufescens', 'Cupania cinerea', 'Cecropia insignis']
ds14_english_name = ['Mucherenje','Degame','Pittoniotis trichantha','Panama Rubber Tree','Doctor Oje','Luehea seemannii','Cypre','Espave','Trumpet Tree','Guatteria lucens','White Olive','Tocoyena pittieri','suicide tree','Andiroba','Botarroma','Peine Mono',np.nan,'Miconia borealis','Virola multiflora','Callichlamys latifolia','Goncalo Alves','Needle Flower Tree','Linguam','Wild Cane','Cojoba rufescens','Portuguese','Guarumo tree']
ds14_pft = ['Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests',np.nan,'Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Shrublands','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests','Evergreen broadleaf forests']

for i in range(len(ds14_traits['Latin Name'].unique())):
    ds14_traits.loc[ds14_traits['Latin Name'] == ds14_traits['Latin Name'].unique()[i], 'Latin Name'] = ds14_latin_name[i]
    ds14_traits.loc[ds14_traits['Latin Name'] == ds14_traits['Latin Name'].unique()[i], 'English Name'] = ds14_english_name[i]
    ds14_traits.loc[ds14_traits['Latin Name'] == ds14_traits['Latin Name'].unique()[i], 'PFT'] = ds14_pft[i]

ds14_traits['LMA'] = ds14_traits['LMA']/10000

ds14_traits["Dataset ID"] = 'Dataset#14'
ds14_traits["Site ID"] = 'Site#53'
ds14_traits['Dataset Name'] = 'NGEE Tropics Leaf Spectral Reflectance Measured in Panama Collected February to April 2016'
ds14_traits["Foreoptic type"] = "Leaf clip"
ds14_traits["Instruments"] = "SVC HR-2014i"
ds14_traits["Reference"] = "(Serbin et al., 2019; Wu et al., 2019)"
ds14_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-tropics-leaf-spectral-reflectance-measured-in-panama-collected-february-to-april-2016"
ds14_traits["DOI"] = "https://doi.org/http://dx.doi.org/10.15486/ngt/1478523"
ds14_traits = ds14_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
ds14_traits = ds14_traits[ds14_traits["LMA"]>0]
ds14_reflectance = ds14_reflectance.iloc[ds14_traits.index]
ds14_reflectance.reset_index(drop = True, inplace = True)
ds14_traits.reset_index(drop = True, inplace = True)

ds14_reflectance = ds14_reflectance.loc[:,"450":"2400"]
ds14_reflectance = do_resample(ds14_reflectance)
ds14 = pd.concat([ds14_reflectance, ds14_traits],axis = 1)
# ds14

### 15. NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests

In [17]:
df8_refl = pd.read_csv('../1_datasets/0_original/15_NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests/nasa_fft_lc_refl_spectra_v4.csv')
df8_refl_meta = pd.read_csv('../1_datasets/0_original/15_NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests/nasa_fft_lc_refl_spectra_v4_metadata.csv')
df8_refl_merge = pd.merge(df8_refl,df8_refl_meta,how = 'left',on = 'SampleID')

df8_traits1 = pd.read_csv('../1_datasets/0_original/15_NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests/nasa_fft_project_cn_data_for_ecosis.csv')
df8_traits2 = pd.read_csv('../1_datasets/0_original/15_NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests/nasa_fft_sla_lma_data_for_ecosis.csv')


df8_all = pd.merge(df8_refl_merge,df8_traits1,how = 'left', on = 'SampleID')
df8_all = pd.merge(df8_all,df8_traits2,how = 'left', on = 'SampleID')

ds15_reflectance = df8_all.loc[:,'350':'2500']
ds15_traits = df8_all.loc[:,'Spectra_Name':]
ds15_traits.rename(columns={'Species': 'usda_code','EWT_gDW_cm2':'EWT','LMA_gDW_cm2':'LMA','Sample_Year':'Sample date'},inplace = True)

ds15_latin_name = ['Acacia rubida', 'Pinus resinosa Aiton', 'Pinus strobus L.', 'Poa gracillima Vasey', 'Quercus rubra L.', 'Abies balsamea (L.)', 'Picea rubens Sarg.', 'Tsuga canadensis (L.)', 'Thalictrum occidentale', 'Besseya alpina', 'Fagus grandifolia', 'Acacia smallii Isely', 'Fraxinus americana L.', 'Ostrya virginiana', 'Tilia americana L.', 'Betula papyrifera', 'Quercus alba L.', 'Camissonia ovata', 'Prenanthes serpentaria', 'Poa tracyi Vasey', 'Polycarpon depressum', 'Ulmus americana L.', 'Fraxinus pennsylvanica', 'Acacia saligna', 'Rhamnus californica', 'Pinus sylvestris L.', 'Juglans nigra L.', 'Lobelia niihauensis and Ceanothus ramulosus', 'Acer platanoides L.', 'Celtis occidentalis L.', 'Quercus ×macnabiana', 'Robinia pseudoacacia L.', 'Acalypha neomexicana', 'Juniperus virginiana L.', 'Proserpinaca pectinata', 'Liriodendron tulipifera L.', 'Quercus ellipsoidalis', 'Betula lenta L.', 'Quercus prinoides', 'Nyssa sylvatica', 'Picea pungens', 'Conyza floribunda', 'Pinguicula villosa L.', 'Callirhoe alcaeoides', 'Rhexia mariana L.', 'Hackelia virginiana', 'Magnolia acuminata (L.) L.', 'Kalmia latifolia L.', 'Salix alaxensis', 'Fraxinus nigra', 'Acamptopappus sphaerocephalus', 'Conopholis americana (L.)', 'Pinus balfouriana ', 'Betula glandulosa', 'Ledum groenlandicum', 'Picea mariana', 'Allionia incarnata L.', 'Larix laricina', 'Spartina pectinata', 'Salix L.', 'Andropogon gerardii', 'Carex L.', 'Phacelia argentea', 'Typha latifolia L.', 'Prunus L.', 'UNKN1']
ds15_english_name = ['Red-leaf wattle', 'Red pine', 'Eastern white pine', 'Sandberg bluegrass', 'Northern red oak', 'Balsam fir', 'Red spruce', 'Eastern hemlock', 'Western meadow-rue', 'Alpine besseya', 'American beech', 'Sweet acacia', 'White ash', 'Hophornbeam', 'American basswood', 'Paper birch', 'White oak', 'Goldeneggs', 'Cankerweed', "Tracy's bluegrass", 'California manyseed', 'American elm', 'Green ash', 'Orange wattle', 'California buckthorn', 'Scots pine', 'Black walnut', 'Niihau lobelia and Buckbrush', 'Norway maple', 'Common hackberry', 'MacNab oak', 'Black locust', 'New Mexico copperleaf', 'Eastern redcedar', 'Comb-leaf mermaidweed', 'Tuliptree', 'Northern pin oak', 'Sweet birch', 'Dwarf chinquapin oak', 'Blackgum', 'Blue spruce', 'Asthmaweed', 'Hairy butterwort\t', 'Light poppymallow', 'Maryland meadowbeauty', 'Beggarslice', 'Cucumber tree', 'Mountain laurel', 'Feltleaf willow', 'Black ash', 'Rayless goldenhead', 'American cancer-root', 'Foxtail pine', 'Resin birch', 'Bog Labrador tea', 'Black spruce', 'Trailing windmills', 'Tamarack', 'Prairie cordgrass', 'Willow', 'Big bluestem', 'Sedge', 'Sanddune phacelia', 'Broadleaf cattail', 'Plum', 'UNKN1']
ds15_pft = ['Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Evergreen needleleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Evergreen needleleaf forests', 'Shrublands', 'Shrublands', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous needleleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands']

for i in range(len(ds15_traits['usda_code'].unique())):
    ds15_traits.loc[ds15_traits['usda_code'] == ds15_traits['usda_code'].unique()[i], 'Latin Name'] = ds15_latin_name[i]
    ds15_traits.loc[ds15_traits['usda_code'] == ds15_traits['usda_code'].unique()[i], 'English Name'] = ds15_english_name[i]
    ds15_traits.loc[ds15_traits['usda_code'] == ds15_traits['usda_code'].unique()[i], 'PFT'] = ds15_pft[i]

ds15_traits["Dataset ID"] = 'Dataset#15'
k = 54
s = ['Adirondacks', 'Baraboo Hills', 'Green Ridge','IDS','Kettle Moraine State Forest','Finland SF / Superior NF',
     'Flambeau River SF / Cheq. NF','Ottawa NF / Sylvania Wilderness','Bayfield / Pine Barrens','Porcupine Mountains',
     'Ely / Superior NF']
for i in s:
    ds15_traits.loc[ds15_traits['Site_Name'] ==i,"Site ID"] = 'Site#'+str(k)  
    k = k+1
    
ds15_traits.loc[ds15_traits['Site_Name'] =='Blackhawk Island',"Site ID"] = 'Site#42'
ds15_traits.loc[ds15_traits['Site_Name'] =='Madison / Dane County',"Site ID"] = 'Site#33'
ds15_traits.loc[ds15_traits['Site_Name'] =='Madison',"Site ID"] = 'Site#33'
ds15_traits['Dataset Name'] = 'NASA FFT Project Leaf Reflectance Morphology and Biochemistry for Northern Temperate Forests'
ds15_traits["Foreoptic type"] = "Integrating sphere"
ds15_traits["Instruments"] = np.nan
ds15_traits["Reference"] = "(Serbin & Townsend, 2014; Serbin et al., 2019)"
ds15_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/nasa-fft-project-leaf-transmittance-morphology-and-biochemistry-for-northern-temperate-forests"
ds15_traits["DOI"] = "https://doi.org/10.1111/nph.16123"
ds15_traits = ds15_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds15_traits = ds15_traits[(ds15_traits["EWT"]>0)|(ds15_traits["LMA"]>0)]
ds15_reflectance = ds15_reflectance.iloc[ds15_traits.index]
ds15_reflectance.reset_index(drop = True, inplace = True)
ds15_traits.reset_index(drop = True, inplace = True)

ds15_reflectance = ds15_reflectance.loc[:,"450":"2400"]
ds15_reflectance = do_resample(ds15_reflectance)
ds15 = pd.concat([ds15_reflectance, ds15_traits],axis = 1)
# ds15

### 16. 2018 Talladega National Forest_Leaf level Reflectance Spectra and Foliar Traits

In [18]:
ds16_reflectance = pd.read_csv('../1_datasets/0_original/16_2018 Talladega National Forest_Leaf level Reflectance Spectra and Foliar Traits/tall_2018_ecosis_spectra.csv')
ds16_reflectance.drop(['Sample_ID'],axis = 1, inplace = True)
ds16_reflectance = ds16_reflectance/100.0
# ds16_reflectance = data_interpolate(ds32_reflectance)

ds16_traits = pd.read_csv('../1_datasets/0_original/16_2018 Talladega National Forest_Leaf level Reflectance Spectra and Foliar Traits/tall_2018_ecosis_spectra_metadata_v2.csv')
ds16_traits.rename(columns={'Latitude':'latitude','Longitude':'longitude','N':'Nmass(%)','Date':'Sample date'},inplace = True)

latitude = []
longitude = []
for i in range(len(ds16_traits)):
    X = ds16_traits['latitude'][i]
    Y = ds16_traits['longitude'][i]
    p = Proj(proj="utm", zone= 16, ellps="WGS84")
    lon,lat = p(X, Y, inverse=True)
    lon, lat = round(lon, 6), round(lat, 6)
    latitude.append(lat)
    longitude.append(lon)

ds16_traits['Latitude'] = np.array(latitude)
ds16_traits['Longitude'] = np.array(longitude)
ds16_traits['Latin Name'] = ds16_traits[['Genus', 'Species']].apply(' '.join, axis=1)

ds16_english_name = ['Longleaf pine','Southern red oak','Mockernut hickory','Tulip tree','Sweetgum','White oak','Loblolly pine','Pignut hickory','Chestnut oak','Blackjack oak']
ds16_pft = ['Evergreen needleleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Evergreen needleleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests','Deciduous broadleaf forests']

for i in range(len(ds16_traits['Latin Name'].unique())):
    ds16_traits.loc[ds16_traits['Latin Name'] == ds16_traits['Latin Name'].unique()[i], 'English Name'] = ds16_english_name[i]
    ds16_traits.loc[ds16_traits['Latin Name'] == ds16_traits['Latin Name'].unique()[i], 'PFT'] = ds16_pft[i]

ds16_traits['LMA'] = ds16_traits['LMA']/10000
ds16_traits["Dataset ID"] = 'Dataset#16'
ds16_traits["Site ID"] = 'Site#17'
ds16_traits['Dataset Name'] = '2018 Talladega National Forest_Leaf level Reflectance Spectra and Foliar Traits'
ds16_traits["Foreoptic type"] = "Leaf clip"
ds16_traits["Instruments"] = "SVC HR-1024i"
ds16_traits["Reference"] = "(Serbin et al., 2017)"
ds16_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/2018-talladega-national-forest--leaf-level-reflectance-spectra-and-foliar-traits"
ds16_traits["DOI"] = "https://doi.org/10.21232/kRB5Gjgi"
ds16_traits = ds16_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
ds16_traits = ds16_traits[ds16_traits["LMA"]>0]
ds16_reflectance = ds16_reflectance.iloc[ds16_traits.index]
ds16_reflectance.reset_index(drop = True, inplace = True)
ds16_traits.reset_index(drop = True, inplace = True)

ds16_reflectance = ds16_reflectance.loc[:,"450":"2400.5"]
ds16_reflectance = do_resample(ds16_reflectance)
ds16 = pd.concat([ds16_reflectance, ds16_traits],axis = 1)
# ds16

### 17. NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska

In [19]:
df9_refl = pd.read_csv('../1_datasets/0_original/17_NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska/sewpen_2019_leaf_spectral_reflectance.csv')
df9_refl_meta = pd.read_csv('../1_datasets/0_original/17_NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska/sewpen_2019_leaf_spectral_reflectance_metadata.csv')
df9_refl_merge = pd.merge(df9_refl, df9_refl_meta, how = 'left', on ='SampleID')

df9_traits1 = pd.read_csv('../1_datasets/0_original/17_NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska/sewpen_2019_cn.csv')
df9_traits2 = pd.read_csv('../1_datasets/0_original/17_NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska/sewpen_2019_lma.csv')

df9_all = pd.merge(df9_refl_merge, df9_traits1, how = 'left', on ='SampleID')
df9_all = pd.merge(df9_all, df9_traits2, how = 'left', on ='SampleID')

ds17_reflectance = df9_all.loc[:,'350':'2500']
ds17_reflectance = ds17_reflectance/100.0

ds17_traits = df9_all.loc[:,'Site':]
ds17_traits.rename(columns={'Species':'usda_code','Date':'Sample date'},inplace = True)

ds17_latin_name = ['Salix pulchra','Alnus viridis','Rubus chamaemorus L.','Betula nana L.','Vaccinium uliginosum L.','Betula glandulosa','Salix richardsonii','Arctostaphylos alpina (L.)','Polygonum bistorta L.','Betula pumila L.','Salix glauca L.']
ds17_english_name = ['Tealeaf willow','Green alder','Cloudberry','Dwarf birch','Bog blueberry','Resin birch',"Richardson's willow", 'Alpine bearberry', 'Meadow bistort', 'Bog birch', 'Grayleaf willow']
ds17_pft = ['Shrublands','Shrublands','Shrublands','Shrublands','Shrublands','Shrublands','Shrublands','Shrublands','Grasslands','Shrublands','Deciduous broadleaf forests']

for i in range(len(ds17_traits['usda_code'].unique())):
    ds17_traits.loc[ds17_traits['usda_code'] == ds17_traits['usda_code'].unique()[i], 'Latin Name'] = ds17_latin_name[i]
    ds17_traits.loc[ds17_traits['usda_code'] == ds17_traits['usda_code'].unique()[i], 'English Name'] = ds17_english_name[i]
    ds17_traits.loc[ds17_traits['usda_code'] == ds17_traits['usda_code'].unique()[i], 'PFT'] = ds17_pft[i]
    
ds17_traits['LMA'] = ds17_traits['LMA']/10000
ds17_traits["Dataset ID"] = 'Dataset#17'
ds17_traits.loc[ds17_traits['Site'] =='Seward_Kougarok',"Site ID"] = 'Site#65'
ds17_traits.loc[ds17_traits['Site'] =='Seward_Teller',"Site ID"] = 'Site#66'
ds17_traits['Dataset Name'] = 'NGEE Arctic 2019 Leaf Spectral Reflectance Seward Peninsula Alaska'
ds17_traits["Foreoptic type"] = "Leaf clip"
ds17_traits["Instruments"] = "SVC HR-1024i"
ds17_traits["Reference"] = "(Serbin et al., 2019)"
ds17_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-arctic-2019-leaf-spectral-reflectance-seward-peninsula-alaska"
ds17_traits["DOI"] = "https://ecosis.org/package/dd3d1e06-9411-4b7a-a3bb-99657e6e9a0c"
ds17_traits = ds17_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]
ds17_traits = ds17_traits[ds17_traits["LMA"]>0]
ds17_reflectance = ds17_reflectance.iloc[ds17_traits.index]
ds17_reflectance.reset_index(drop = True, inplace = True)
ds17_traits.reset_index(drop = True, inplace = True)

ds17_reflectance = ds17_reflectance.loc[:,"450":"2400"]
ds17_reflectance = do_resample(ds17_reflectance)
ds17 = pd.concat([ds17_reflectance, ds17_traits],axis = 1)
# ds17

### 18. NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska

In [20]:
df10_refl = pd.read_csv('../1_datasets/0_original/18_NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska/seward_2017_leaf_spectral_reflectance.csv')
df10_refl_meta = pd.read_csv('../1_datasets/0_original/18_NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska/seward_2017_leaf_spectral_reflectance_metadata.csv')
df10_refl_merge = pd.merge(df10_refl, df10_refl_meta, how = 'left', on ='SampleID')

df10_traits1 = pd.read_csv('../1_datasets/0_original/18_NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska/seward_2017_cn.csv')
df10_traits2 = pd.read_csv('../1_datasets/0_original/18_NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska/seward_2017_leaf_samples_lma.csv')
df10_all = pd.merge(df10_refl_merge, df10_traits1, how = 'left', on ='SampleID')
df10_all = pd.merge(df10_all, df10_traits2, how = 'left', on ='SampleID')

ds18_reflectance = df10_all.loc[:,'350':'2500']
ds18_reflectance = ds18_reflectance/100.0

ds18_traits = df10_all.loc[:,'Site':]

ds18_traits['Latitude'] = 64.7083056
ds18_traits['Longitude'] = -166.0432441

ds18_traits.rename(columns={'Species':'usda_code','LWC':'RWC','Date':'Sample date'},inplace = True)

ds18_latin_name = ['Salix pulchra','Arctagrostis latifolia','Betula nana L.','Carex aquatilis','Dodecatheon frigidum','Equisetum arvense L.','Petasites frigidus (L.)','Rubus chamaemorus L.','Rubus saxatilis L.','Salix alaxensis','Salix glauca L.','Salix reticulata L.','Salix richardsonii','Vaccinium uliginosum L.','Salix lanata L.','Betula glandulosa']
ds18_english_name = ['Tealeaf willow','Wideleaf polargrass','Dwarf birch','Water sedge','Western arctic shootingstar','Field horsetail','Arctic sweet coltsfoot','Cloudberry','Stone bramble','Feltleaf willow','Grayleaf willow','Netleaf willow',"Richardson's willow",'Bog blueberry','Woolly willow','Resin birch']
ds18_pft = ['Shrublands','Grasslands','Shrublands','Grasslands','Grasslands','Grasslands','Grasslands','Shrublands','Shrublands','Shrublands','Deciduous broadleaf forests','Shrublands','Shrublands','Shrublands','Shrublands','Shrublands']

for i in range(len(ds18_traits['usda_code'].unique())):
    ds18_traits.loc[ds18_traits['usda_code'] == ds18_traits['usda_code'].unique()[i], 'Latin Name'] = ds18_latin_name[i]
    ds18_traits.loc[ds18_traits['usda_code'] == ds18_traits['usda_code'].unique()[i], 'English Name'] = ds18_english_name[i]
    ds18_traits.loc[ds18_traits['usda_code'] == ds18_traits['usda_code'].unique()[i], 'PFT'] = ds18_pft[i]
    
ds18_traits['LMA'] = ds18_traits['LMA']/10000
ds18_traits["Dataset ID"] = 'Dataset#18'
ds18_traits["Site ID"] = 'Site#66'
ds18_traits['Dataset Name'] = 'NGEE Arctic 2017 Leaf Spectral Reflectance Teller Watershed Seward Peninsula Alaska'
ds18_traits["Foreoptic type"] = "Leaf clip"
ds18_traits["Instruments"] = "SVC HR-1024i"
ds18_traits["Reference"] = "(Serbin et al., 2017)"
ds18_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-arctic-2017-leaf-spectral-reflectance-teller-watershed-seward-peninsula-alaska"
ds18_traits["DOI"] = "https://ecosis.org/package/b64174d0-c426-4e90-b9ed-c3afdcb8bb73"
ds18_traits = ds18_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds18_traits = ds18_traits[ds18_traits["LMA"]>0]
ds18_reflectance = ds18_reflectance.iloc[ds18_traits.index]
ds18_reflectance.reset_index(drop = True, inplace = True)
ds18_traits.reset_index(drop = True, inplace = True)

ds18_reflectance = ds18_reflectance.loc[:,"450":"2400"]
ds18_reflectance = do_resample(ds18_reflectance)
ds18 = pd.concat([ds18_reflectance, ds18_traits],axis = 1)
# ds18

### 19. Leaf spectra and physiological and chemical traits from maize grown under nitrogen stress
https://plantmethods.biomedcentral.com/articles/10.1186/s13007-019-0450-8

In [21]:
ds19_reflectance = pd.read_csv('../1_datasets/0_original/19_Leaf spectra and physiological and chemical traits from maize grown under nitrogen stress/maize_2018_2019_unl_spectra.csv')
ds19_reflectance = ds19_reflectance.loc[:,'350':]

ds19_traits = pd.read_csv('../1_datasets/0_original/19_Leaf spectra and physiological and chemical traits from maize grown under nitrogen stress/maize_2018_2019_unl_traits.csv')
ds19_traits['Species'] = 'Maize'

ds19_traits['Latitude'] = 45.8636081
ds19_traits['Longitude'] = -96.5191698

ds19_traits.rename(columns={'Species':'English Name','Chlorophyll':'Chla+b','Year':'Sample date'},inplace = True)
ds19_traits['Latin Name'] = 'Zea mays'
ds19_traits['PFT'] = 'Croplands'
## 898.16 g/mol
ds19_traits['Chla+b'] = ds19_traits['Chla+b']*898.16/10000

ds19_traits["Dataset ID"] = 'Dataset#19'
ds19_traits["Site ID"] = 'Site#67'
ds19_traits['Dataset Name'] = 'Leaf spectra and physiological and chemical traits from maize grown under nitrogen stress'
ds19_traits["Foreoptic type"] = "Leaf clip"
ds19_traits["Instruments"] = "ASD FieldSpec 4"
ds19_traits["Reference"] = "(Ge et al., 2019; Grzybowski et al., 2020)"
ds19_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-spectra-and-physiological-and-chemical-traits-from-maize-grown-under-nitrogen-stress"
ds19_traits["DOI"] = "https://doi.org/10.21232/y5TTxY3N"
ds19_traits = ds19_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds19_traits = ds19_traits[ds19_traits["Chla+b"]>0]
ds19_reflectance = ds19_reflectance.iloc[ds19_traits.index]
ds19_reflectance.reset_index(drop = True, inplace = True)
ds19_traits.reset_index(drop = True, inplace = True)

ds19_reflectance = ds19_reflectance.loc[:,"450":"2400"]
ds19_reflectance = do_resample(ds19_reflectance)
ds19 = pd.concat([ds19_reflectance, ds19_traits],axis = 1)
# ds19

### 20. NGEE Arctic Leaf Spectral Reflectance Utqiagvik (Barrow) Alaska 2013

In [22]:
df11_refl = pd.read_csv('../1_datasets/0_original/20_NGEE Arctic Leaf Spectral Reflectance Utqiagvik (Barrow) Alaska 2013/leaf_spectra_barrow_2013_20180824_ecosis.csv')
df11_refl_meta = pd.read_csv('../1_datasets/0_original/20_NGEE Arctic Leaf Spectral Reflectance Utqiagvik (Barrow) Alaska 2013/ngeearctic_bnl_2013_leaf_spectra_traits_metadata.csv')
df11_traits = pd.read_csv('../1_datasets/0_original/20_NGEE Arctic Leaf Spectral Reflectance Utqiagvik (Barrow) Alaska 2013/ngeearctic_bnl_leaf_c_n_lma_2013.csv')

df11_all = pd.merge(df11_refl,df11_refl_meta, how ='left', on = 'Sample_ID')
df11_all = pd.merge(df11_all,df11_traits, how ='left', on = 'Sample_ID')

ds20_reflectance = df11_all.loc[:,'350':'2500']
ds20_traits = df11_all.loc[:,'USDA_Species':]

ds20_traits['Latitude'] = 71.275
ds20_traits['Longitude'] = -156.641

ds20_traits.rename(columns={'USDA_Species':'usda_code','LMA_g_m2':'LMA','Sample_Date':'Sample date'},inplace = True)
ds20_latin_name = ['Salix pulchra','Petasites frigidus (L.)','Saxifraga punctata L.','Arctophila fulva','Dupontia fisheri','Carex aquatilis','Arctagrostis latifolia','Eriophorum angustifolium','Vaccinium vitis-idaea L.']
ds20_english_name = ['Tealeaf willow','Arctic sweet coltsfoot','Dotted Saxifrage','Pendantgrass, Grasslands',"Fisher's tundragrass",'Water sedge','Wideleaf polargrass','Tall cottongrass','lingonberry']
ds20_pft = ['Shrublands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Shrublands']

for i in range(len(ds20_traits['usda_code'].unique())):
    ds20_traits.loc[ds20_traits['usda_code'] == ds20_traits['usda_code'].unique()[i], 'Latin Name'] = ds20_latin_name[i]
    ds20_traits.loc[ds20_traits['usda_code'] == ds20_traits['usda_code'].unique()[i], 'English Name'] = ds20_english_name[i]
    ds20_traits.loc[ds20_traits['usda_code'] == ds20_traits['usda_code'].unique()[i], 'PFT'] = ds20_pft[i]

ds20_traits['LMA'] = ds20_traits['LMA']/10000

ds20_traits["Dataset ID"] = 'Dataset#20'
ds20_traits["Site ID"] = 'Site#68'
ds20_traits['Dataset Name'] = 'NGEE Arctic Leaf Spectral Reflectance Utqiagvik (Barrow) Alaska 2013'
ds20_traits["Foreoptic type"] = "Leaf clip"
ds20_traits["Instruments"] = "ASD FieldSpec 3"
ds20_traits["Reference"] = "(Serbin et al., 2018)"
ds20_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-arctic-leaf-spectral-reflectance-utqiagvik--barrow--alaska-2013"
ds20_traits["DOI"] = "https://doi.org/10.5440/1441203"
ds20_traits = ds20_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds20_reflectance = ds20_reflectance.loc[:,"450":"2400"]
ds20_reflectance = do_resample(ds20_reflectance)
ds20 = pd.concat([ds20_reflectance, ds20_traits],axis = 1)
# ds20

### 21. NGEE Arctic 2016 Leaf Spectral Reflectance Kougarok Road Watershed Seward Peninsula Alaska

In [23]:
df12_refl = pd.read_csv('../1_datasets/0_original/21_NGEE Arctic 2016 Leaf Spectral Reflectance Kougarok Road Watershed Seward Peninsula Alaska/ngee-arctic_seward_2016_hr1024i_leaf_spectral_reflectance.csv')
df12_refl_meta =pd.read_csv('../1_datasets/0_original/21_NGEE Arctic 2016 Leaf Spectral Reflectance Kougarok Road Watershed Seward Peninsula Alaska/ngee-arctic_seward_2016_svchr1024i_leaf_spectral_reflectance_metadata.csv')
df12_traits = pd.read_csv('../1_datasets/0_original/21_NGEE Arctic 2016 Leaf Spectral Reflectance Kougarok Road Watershed Seward Peninsula Alaska/ngeearctic_bnl_seward_lma_cn_2016.csv')

df12_all = pd.merge(df12_refl,df12_refl_meta, how ='left', on = 'Sample_ID')
df12_all = pd.merge(df12_all,df12_traits, how ='left', on = 'Sample_ID')

ds21_reflectance = df12_all.loc[:,'350':'2500']
ds21_reflectance = ds21_reflectance/100.0
ds21_traits = df12_all.loc[:,'Site':]

ds21_traits.rename(columns={'Spectrometer_GPS_Lat':'Latitude','Spectrometer_GPS_Long':'Longitude',
                            'USDA_Species_Code':'usda_code','LMA_g_m2':'LMA','Sample_Date':'Sample date'},inplace = True)

ds21_latin_name = ['Alnus viridis','Salix pulchra','Salix richardsonii', 'Betula glandulosa', 'Salix glauca L.']
ds21_english_name = ['Siberian alder','Tealeaf willow', "Richardson's willow",'Resin birch','Grayleaf willow']
ds21_pft = ['Shrublands','Shrublands','Shrublands','Shrublands','Deciduous broadleaf forests']

for i in range(len(ds21_traits['usda_code'].unique())):
    ds21_traits.loc[ds21_traits['usda_code'] == ds21_traits['usda_code'].unique()[i], 'Latin Name'] = ds21_latin_name[i]
    ds21_traits.loc[ds21_traits['usda_code'] == ds21_traits['usda_code'].unique()[i], 'English Name'] = ds21_english_name[i]
    ds21_traits.loc[ds21_traits['usda_code'] == ds21_traits['usda_code'].unique()[i], 'PFT'] = ds21_pft[i]

ds21_traits['LMA'] = ds21_traits['LMA']/10000
ds21_traits["Dataset ID"] = 'Dataset#21'
ds21_traits["Site ID"] = 'Site#69'
ds21_traits['Dataset Name'] = 'NGEE Arctic 2016 Leaf Spectral Reflectance Kougarok Road Watershed Seward Peninsula Alaska'
ds21_traits["Foreoptic type"] = "Leaf clip"
ds21_traits["Instruments"] = "SVC HR-1024i"
ds21_traits["Reference"] = "(Serbin & Rogers, 2016)"
ds21_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/ngee-arctic-2016-leaf-spectral-reflectance-kougarok-road-watershed-seward-peninsula-alaska"
ds21_traits["DOI"] = "https://doi.org/https://doi.org/10.5440/1430080"
ds21_traits = ds21_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds21_reflectance = ds21_reflectance.loc[:,"450":"2400"]
ds21_reflectance = do_resample(ds21_reflectance)
ds21 = pd.concat([ds21_reflectance, ds21_traits],axis = 1)
# ds21

### 22. Leaf spectra of 36 species growing in Rosa rugosa invaded coastal grassland communities in Belgium

In [24]:
ds22_reflectance = pd.read_csv('../1_datasets/0_original/22_Leaf spectra of 36 species growing in Rosa rugosa invaded coastal grassland communities in Belgium/spectra.csv')
ds22_reflectance = ds22_reflectance.loc[:,'350':]

ds22_traits = pd.read_csv('../1_datasets/0_original/22_Leaf spectra of 36 species growing in Rosa rugosa invaded coastal grassland communities in Belgium/traits.csv')

ds22_traits['Latitude'] = 51.2269051
ds22_traits['Longitude'] = 3.0392025

ds22_traits.rename(columns={'species':'Latin Name','Leaf area (mm2)':'Area','LMA (g/cm2)':'LMA',
                            'Cw/EWT (cm3/cm2)':'EWT','Leaf N (mg/mm2)':'Narea','Leaf P (mg/mm2)':'P_area',
                            'Leaf Ca (mg/mm2)':'Ca_area','Leaf K (mg/mm2)':'K_area',
                            'Leaf Mg (mg/mm2)':'Mg_area'},inplace = True)

ds22_english_name = ['bulbous oat grass','Barren brome','Ragwort','European dewberry','Smooth hawksbeard','Canadian hawkweed','Sand sedge','Sea-buckthorn','Russian wheatgrass','Large-flowered evening-primrose','Rugosa rose','Catsear','Annual Ryegrass','Red fescue','Creeping cinquefoil','Marram grass','Ribwort plantain',"Cock's-foot",'Common restharrow','Whitetop','Cleavers','Yorkshire fog','Herb-Robert','Sea bindweed','Timothy-grass','Hairy sedge','Honeysuckle', 'Curly dock','Esser meadow-rue','Narrow-leaved ragwort','Ground-ivy','Yellow rattle','Common nettle','White willow','Sand ryegrass','Spindle']
ds22_pft = ['Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Shrublands','Grasslands','Grasslands','Shrublands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Grasslands','Vine','Grasslands','Grasslands','Deciduous broadleaf forests','Grasslands','Shrublands']

for i in range(len(ds22_traits['Latin Name'].unique())):
    ds22_traits.loc[ds22_traits['Latin Name'] == ds22_traits['Latin Name'].unique()[i], 'English Name'] = ds22_english_name[i]
    ds22_traits.loc[ds22_traits['Latin Name'] == ds22_traits['Latin Name'].unique()[i], 'PFT'] = ds22_pft[i]

ds22_traits["Dataset ID"] = 'Dataset#22'
ds22_traits["Site ID"] = 'Site#70'
ds22_traits['Dataset Name'] = 'Leaf spectra of 36 species growing in Rosa rugosa invaded coastal grassland communities in Belgium'
ds22_traits["Foreoptic type"] = "Leaf clip"
ds22_traits["Instruments"] = "SVC HR-1024tm"
ds22_traits["Reference"] = "(Helsen et al., 2020)"
ds22_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-spectra-of-36-species-growing-in-rosa-rugosa-invaded-coastal-grassland-communities-in-belgium"
ds22_traits["DOI"] = "https://doi.org/doi:10.21232/9nr6-sq54"
ds22_traits = ds22_traits[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT",
                         "Latitude","Longitude","EWT","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds22_reflectance = ds22_reflectance.loc[:,"450":"2400"]
ds22_reflectance = do_resample(ds22_reflectance)
ds22 = pd.concat([ds22_reflectance, ds22_traits],axis = 1)
# ds22

### 23. Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI

In [25]:
ds23_refl_asd = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_asd_spectra.csv')
ds23_refl_psr = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_psr_spectra.csv')
ds23_refl_svc = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_svc_spectra.csv')

ds23_refl_asd.drop(['id'],axis = 1, inplace = True)
ds23_refl_psr.drop(['id'],axis = 1, inplace = True)
ds23_refl_svc.drop(['id'],axis = 1, inplace = True)

ds23_refl_asd.columns = ds23_refl_asd.columns.astype(float).astype(int).astype(str)
ds23_refl_psr.columns = ds23_refl_psr.columns.astype(float).astype(int).astype(str)
ds23_refl_svc.columns = ds23_refl_svc.columns.astype(float).astype(int).astype(str)
ds23_reflectance = pd.concat([ds23_refl_asd,ds23_refl_psr,ds23_refl_svc], axis = 0)


ds23_traits_asd = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_asd_lma_g_m2.csv')
ds23_traits_psr = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_psr_lma_g_m2.csv')
ds23_traits_svc = pd.read_csv('../1_datasets/0_original/23_Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI/lt_svc_lma_g_m2.csv')

ds23_traits_asd["Instruments"] = "ASD FieldSpec 3"
ds23_traits_psr["Instruments"] = "PSR 3500+"
ds23_traits_svc["Instruments"] = "SVC HR-1024i"
ds23_traits = pd.concat([ds23_traits_asd,ds23_traits_psr,ds23_traits_svc], axis = 0)

ds23_traits['Latitude'] = 43.0697829
ds23_traits['Longitude'] = -89.4415172
ds23_traits.rename(columns={'functional_type':'PFT','lma_g_m2':'LMA'},inplace = True)
ds23_traits['LMA'] = ds23_traits['LMA']/10000

ds23_traits.loc[ds23_traits['PFT'] == 'tree','PFT'] = 'Deciduous broadleaf forests'
ds23_traits.loc[ds23_traits['PFT'] == 'forb','PFT'] = 'Grasslands'
ds23_traits.loc[ds23_traits['PFT'] == 'grass','PFT'] = 'Grasslands'
ds23_traits.loc[ds23_traits['PFT'] == 'vine','PFT'] = 'Vine'

ds23_traits["Dataset ID"] = 'Dataset#23'
ds23_traits['Site ID'] = 'Site#33'
ds23_traits['Dataset Name'] = 'Leaf level spectra and LMA for a set of trees, forbs, vines and grasses collected in Madison, WI'
ds23_traits["Foreoptic type"] = "Leaf clip"
ds23_traits["Reference"] = "(Chlus, 2019)"
ds23_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-level-spectra-and-lma-for-a-set-of-trees--forbs--vines-and-grasses-collected-in-madison--wi"
ds23_traits["DOI"] = "https://ecosis.org/package/7433af7d-fbbd-4617-8df4-4d892f0d4357"
ds23_traits = ds23_traits[["Dataset ID","Dataset Name","Site ID","PFT",
                         "Latitude","Longitude","LMA","Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds23_traits.reset_index(drop = True, inplace = True)
ds23_reflectance.reset_index(drop = True, inplace = True)
ds23_reflectance = ds23_reflectance.loc[:,"450":"2400"]

ds23_reflectance = do_resample(ds23_reflectance)
ds23 = pd.concat([ds23_reflectance, ds23_traits],axis = 1)
# ds23

### 24. Fresh Leaf Spectra to Estimate Foliar Functional Traits over NEON domains in eastern United States

In [26]:
df13_refl = pd.read_csv('../1_datasets/0_original/24_Fresh Leaf Spectra to Estimate Foliar Functional Traits over NEON domains in eastern United States/neon_fresh_spectra.csv')
df13_refl_meta = pd.read_csv('../1_datasets/0_original/24_Fresh Leaf Spectra to Estimate Foliar Functional Traits over NEON domains in eastern United States/neon_fresh_metadata.csv')
df13_traits = pd.read_csv('../1_datasets/0_original/24_Fresh Leaf Spectra to Estimate Foliar Functional Traits over NEON domains in eastern United States/neon_fresh_traits.csv')

df13_all = pd.merge(df13_refl,df13_refl_meta,how = 'left', on = 'Sample_ID')
df13_all = pd.merge(df13_all,df13_traits,how = 'left', on = 'Sample_ID')
df13_all = df13_all[df13_all['350'].notnull()]

ds24_reflectance = df13_all.loc[:,'350':'2500']
ds24_traits = df13_all.loc[:,'Sample_Date':]

ds24_traits['Latitude'] = 0
ds24_traits['Longitude'] = 0

for i in ds24_traits['Site_Name'].unique():
    geolocator = Nominatim(user_agent='BuyiXiao',timeout=5)
    location = geolocator.geocode(i)
    if location is None:
        pass
    else:
        ds24_traits.loc[ds24_traits['Site_Name'] == i, 'Latitude'] = location.latitude
        ds24_traits.loc[ds24_traits['Site_Name'] == i, 'Longitude'] = location.longitude
        
ds24_traits.loc[ds24_traits['Site_Name'] == 'Konza Prairie Biological Station', 'Latitude'] = 39.0546
ds24_traits.loc[ds24_traits['Site_Name'] == 'Konza Prairie Biological Station', 'Longitude'] = -96.7307

ds24_traits.loc[ds24_traits['Site_Name'] == 'The University of Kansas Field Station', 'Latitude'] = 38.9822
ds24_traits.loc[ds24_traits['Site_Name'] == 'The University of Kansas Field Station', 'Longitude'] = -95.2877

ds24_traits.loc[ds24_traits['Site_Name'] == 'Northern Great Plains Research Laboratory', 'Latitude'] = 46.7697
ds24_traits.loc[ds24_traits['Site_Name'] == 'Northern Great Plains Research Laboratory', 'Longitude'] = -100.9154

ds24_traits.loc[ds24_traits['Site_Name'] == 'Jones Ecological Research Center', 'Latitude'] = 31.2202045
ds24_traits.loc[ds24_traits['Site_Name'] == 'Jones Ecological Research Center', 'Longitude'] = -84.4789307

ds24_traits.loc[ds24_traits['Site_Name'] == 'UNDERC', 'Latitude'] = 46.2339 
ds24_traits.loc[ds24_traits['Site_Name'] == 'UNDERC', 'Longitude'] = -89.5373

ds24_traits.loc[ds24_traits['Site_Name'] == 'Lenoir Landing', 'Latitude'] = 32.1237024
ds24_traits.loc[ds24_traits['Site_Name'] == 'Lenoir Landing', 'Longitude'] = -88.2464611

ds24_traits.rename(columns={'USDA Symbol':'usda_code','Chlorophylls_area':'Chla+b',
                            'Carotenoids_area':'Ccar','Sample_Date':'Sample date'},inplace = True)

ds24_latin_name = ['Quercus coccinea Münchh.', 'Madia madioides(Nutt.) Greene', 'Salix nigra', 'Macromeria viridiflora', 'Oxydendrum arboreum(L.) DC.', 'Conyza floribunda', 'Quercus stellata', 'Nymphaea sagittifolia', 'Calla palustris', 'Quercus nigra', 'Albizia julibrissin', 'Quercus lyrata', 'Carex illota', 'Quercus acuminata(Michx.) Houba', 'Quercus ×palaeolithicola', 'Celtis laevigata', 'Nymphoides aquatica (J.F. Gmel.) Kuntze', 'Juglans cinerea', 'Calliandra conferta', 'Hackelia diffusa(Lehm.) I.M. Johnst.', 'Ulmus americana', 'Distichlis spicata (L.) Greene', 'Rhus glabra', 'Quercus muehlenbergii Engelm.', 'Gleditsia triacanthos', 'Syringa vulgaris', 'Ailanthus altissima(Mill.) Swingle', 'Elaeagnus angustifolia', 'Salix L.', 'Malus spp.', 'Quercus velutina', 'Vitis riparia', 'Asimina triloba(L.) Dunal', 'Campanula glomerata', 'Magnolia acuminata(L.) L.', 'Betula lenta', 'Proserpinaca pectinata', 'Quercus mohriana', 'Palafoxia rosea(Bush) Cory', 'Calochortus tolmiei', 'Linum striatum', 'Quercus falcata', 'cer saccharum', 'Poa gracillima', 'Quercus macrocarpa', 'Quercus laceyi', 'Quercus phellos', 'Quercus shumardii', 'Melia azedarach', 'Acer floridanum(Chapm.) Pax', 'Quercus virginiana Mill.', 'Sabal palmetto(Walter) Lodd. ex Schult. & Schult. f.', 'Persea spp.', 'Carex amphibola', 'Spartina pectinata', 'Koeleria macrantha(Ledeb.) Schult.', 'Andropogon gerardii Vitman', 'Trichomanes davallioides', 'Panicum virgatum', 'Pascopyrum smithii', 'Sophora nuttalliana', 'Schizachyrium scoparium(Michx.) Nash', np.nan, 'Phacelia argentea', 'Typha latifolia', 'Phacelia austromontana', 'Poa pratensis L.', 'Solidago L.', 'Amaranthus arenicola', 'Teucrium laciniatum', 'Apocynum cannabinum', 'Ambrosia trifida', 'Medicago sativa L.', 'Sicyos laciniatus', 'Helianthus L.', 'Rhus copallinum', 'Cornus drummondii', 'Shepherdia argentea(Pursh) Nutt.', 'Rhamnus californica', 'Maranthes corymbosa', 'Rubus allegheniensis', 'Allionia incarnata', 'Sedum reflexum', 'Quercus michauxii', 'Pinus taeda', 'Tabernaemontana divaricata(L.) R.', 'Juniperus virginiana', 'Pinus sylvestris', 'Picea pungens', 'Pinus strobus', 'Tsuga canadensis(L.) Carrière', 'Pinus rigida', 'Thalictrum occidentale', 'Larix laricina(Du Roi) K. Koch', 'Abies balsamea(L.) Mill.', 'Picea mariana(Mill.) Britton, Sterns & Poggenb.', 'Pinus balfouriana', 'Pinus resinosa', 'Picea glauca(Moench) Voss', 'Pinus glabra', 'Picris echioides', 'Pinaropappus parvus', 'Talinum calycinum', 'Picea abies(L.) Karst.', 'Abutilon berlandieri', 'Juncus nevadensis', 'Lagenaria siceraria', 'Pilea nummulariifolia', 'Labordia kaalae', 'Pinus koraiensis', 'Pseudotsuga menziesii']
ds24_english_name = ['Scarlet oak', 'Woodland madia', 'Black willow', 'Giant-trumpets', 'Sourwood', 'Asthmaweed', 'Post oak', 'Yellow pond-lily', 'Water arum', 'Water oak', 'Persian Silk Tree', 'Overcup oak', 'Sheep sedge', 'Mapleleaf oak', 'Hybrid oak', 'Sugarberry', 'Big floatingheart', 'Butternut', 'Rio Grande stickpea', 'Spreading stickseed', 'American elm', 'Saltgrass', 'Smooth sumac', 'Chinquapin oak', 'Honeylocust', 'Common lilac', 'Tree of heaven', 'Russian olive', 'Willow', 'Crabapple', 'Black oak', 'Riverbank grape', 'Pawpaw', "Dane's blood", 'Cucumber tree', 'Sweet birch', 'Combleaf mermaidweed', 'Mohr oak', 'Rosy palafox', 'Tolmie star-tulip', 'Ridged yellow flax', 'Southern red oak', 'Black maple', 'Slender bluegrass', 'Bur oak', 'Lacey oak', 'Willow oak', "Shumard's oak", 'Chinaberrytree', 'Southern sugar maple', 'Live oak', 'Cabbage palmetto', 'Avocado pear', 'Creek Sedge', 'Prairie cordgrass', 'Prairie Junegrass', 'Big bluestem', 'Kilau', 'Switchgrass', 'Western wheatgrass', 'Silky sophora', 'Little bluestem', np.nan, 'Sanddune phacelia', 'Broadleaf cattail', 'Southern Sierra phacelia', 'Kentucky bluegrass', 'Goldenrod', 'Sandhill amaranth', 'Lacy germander', 'Indianhemp', 'Great ragweed', 'Alfalfa', 'Cutleaf bur cucumber', 'Sunflower', 'Winged sumac', 'Roughleaf dogwood', 'Silver buffaloberry', 'California buckthorn', 'Bkau', 'Allegheny blackberry', 'Trailing windmills', "Jenny's stonecrop", 'Swamp chestnut oak', 'Loblolly pine', 'Pinwheelflower', 'Eastern redcedar', 'Scots pine', 'Blue spruce', 'Eastern white pine', 'Eastern hemlock', 'Pitch pine', 'Western meadow-rue', 'Tamarack', 'Balsam fir', 'Black spruce', 'Foxtail pine', 'Red pine', 'White spruce', 'Spruce pine', 'Bristly oxtongue', 'Small rocklettuce', 'Largeflower fameflower', 'Norway spruce', "Berlandier's Indian mallow", 'Sierra rush', 'Bottle gourd', 'Creeping charlie', 'Waianae Range labordia', 'Chinese pinenut', 'Douglas-fir']
ds24_pft = ['Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Vine', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', np.nan, 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Grasslands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Evergreen broadleaf forests', 'Shrublands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Deciduous needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Grasslands', 'Grasslands', 'Shrublands', 'Evergreen needleleaf forests', 'Shrublands', 'Grasslands', 'Vine', 'Grasslands', 'Shrublands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests']

for i in range(len(ds24_traits['usda_code'].unique())):
    ds24_traits.loc[ds24_traits['usda_code'] == ds24_traits['usda_code'].unique()[i], 'Latin Name'] = ds24_latin_name[i]
    ds24_traits.loc[ds24_traits['usda_code'] == ds24_traits['usda_code'].unique()[i], 'English Name'] = ds24_english_name[i]
    ds24_traits.loc[ds24_traits['usda_code'] == ds24_traits['usda_code'].unique()[i], 'PFT'] = ds24_pft[i]

ds24_traits["Dataset ID"] = 'Dataset#24'
ds24_traits.loc[ds24_traits['Site_Name']=='Talladega National Forest','Site ID'] = 'Site#17'
ds24_traits.loc[ds24_traits['Site_Name']=='Dead Lake','Site ID'] = 'Site#18'
ds24_traits.loc[ds24_traits['Site_Name']=='Oak Ridge National Laboratory','Site ID'] = 'Site#19'
ds24_traits.loc[ds24_traits['Site_Name']=='Lenoir Landing','Site ID'] ='Site#20'
ds24_traits.loc[ds24_traits['Site_Name']=='Konza Prairie Biological Station','Site ID'] = 'Site#22'
ds24_traits.loc[ds24_traits['Site_Name']=='The University of Kansas Field Station','Site ID'] = 'Site#23'
ds24_traits.loc[ds24_traits['Site_Name']=='Northern Great Plains Research Laboratory','Site ID'] = 'Site#24'
ds24_traits.loc[ds24_traits['Site_Name']=='Smithsonian Conservation Biology Institute','Site ID'] = 'Site#25'
ds24_traits.loc[ds24_traits['Site_Name']=='Mountain Lake Biological Station','Site ID'] = 'Site#26'
ds24_traits.loc[ds24_traits['Site_Name']=='Smithsonian Environmental Research Center','Site ID'] = 'Site#25'
ds24_traits.loc[ds24_traits['Site_Name']=='Blandy Experimental Farm','Site ID'] = 'Site#27'
ds24_traits.loc[ds24_traits['Site_Name']=='Chequamegon-Nicolet National Forest','Site ID'] = 'Site#28'
ds24_traits.loc[ds24_traits['Site_Name']=='Jones Ecological Research Center','Site ID'] ='Site#29'
ds24_traits.loc[ds24_traits['Site_Name']=='Disney Wilderness Preserve','Site ID'] ='Site#30'
ds24_traits.loc[ds24_traits['Site_Name']=='Woodworth','Site ID'] ='Site#71'
ds24_traits.loc[ds24_traits['Site_Name']== 'UNDERC','Site ID'] ='Site#72'
ds24_traits.loc[ds24_traits['Site_Name']== 'Madison','Site ID'] = 'Site#33'


ds24_traits["Dataset ID"] = 'Dataset#24'
ds24_traits['Dataset Name'] = 'Fresh Leaf Spectra to Estimate Foliar Functional Traits over NEON domains in eastern United States'
ds24_traits["Foreoptic type"] = "Contact probe"
ds24_traits["Instruments"] = "ASD FieldSpec 3"
ds24_traits["Reference"] = "(Wang, 2019)"
ds24_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-spectra-to-estimate-foliar-functional-traits-over-neon-domains-in-eastern-united-states"
ds24_traits["DOI"] = "https://doi.org/doi:10.21232/gx9f-5546"
ds24_traits = ds24_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name","PFT",
                         "Latitude","Longitude","Chla+b","Ccar", "Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds24_traits.reset_index(drop = True, inplace = True)
ds24_reflectance = ds24_reflectance.loc[:,"450":"2400"]
ds24_reflectance = do_resample(ds24_reflectance)
ds24 = pd.concat([ds24_reflectance, ds24_traits],axis = 1)
# ds24

### 25. Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States

In [27]:
df14_refl_asd = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_asd_spectra.csv')
df14_refl_meta_asd = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_asd_metadata.csv')
df14_traits_asd = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_asd_lma.csv')

df14_all_asd = pd.merge(df14_refl_asd,df14_refl_meta_asd,how = 'left', on = 'Sample_ID')
df14_all_asd = pd.merge(df14_all_asd,df14_traits_asd,how = 'left', on = 'Sample_ID')

ds25_reflectance_asd = df14_all_asd.loc[:,'350':'2500']
ds25_traits_asd = df14_all_asd.loc[:,'USDA Symbol':]

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D02', 'Latitude'] = 38.8885357
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D02', 'Longitude'] = -78.1672628

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D03', 'Latitude'] = 29.6989925
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D03', 'Longitude'] = -82.3300268

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D05', 'Latitude'] = 46.2068814
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D05', 'Longitude'] = -89.4843009

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D06', 'Latitude'] = 39.1948221
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D06', 'Longitude'] = -96.5570739

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D07', 'Latitude'] = 36.0294079
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D07', 'Longitude'] = -84.2283144

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D08', 'Latitude'] = 33.1529079
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D08', 'Longitude'] = -87.5503075

ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D09', 'Latitude'] = 46.8936068
ds25_traits_asd.loc[ds25_traits_asd['Domain'] == 'D09', 'Longitude'] = -98.6879072

ds25_traits_asd.rename(columns={'USDA Symbol':'usda_code'},inplace = True)

ds25_asd_latin_name = ['Juglans nigra L.', 'Quercus velutina', 'Prenanthes serpentaria', 'Vitis riparia', 'Platanus occidentalis', 'Digitaria villosa', np.nan, 'Asimina triloba(L.) Dunal', 'Ailanthus altissima(Mill.) Swingle', 'Liriodendron tulipifera', 'Acer rubrum', 'Cornus sp.', 'Calliandra conferta', 'Acalypha neomexicana', 'Quercus rubra', 'Fraxinus americana', 'Linanthus bellus', 'Fraxinus pennsylvanica', 'Panicum torridum', 'Quercus alba', 'Quercus mohriana', 'Campanula glomerata', 'Quercus falcata', 'Fagus grandifolia', 'Calochortus tolmiei', 'Cabomba caroliniana', 'Linum striatum', 'Ulmus americana', 'Nyssa sylvatica', 'cer saccharum', 'Celtis occidentalis L.', 'Robinia pseudoacacia L.', 'Maclura pomifera', 'Ginkgo biloba L.', 'Quercus virginiana Mill.', 'Quercus macrocarpa', 'Quercus ×heterophylla', 'Quercus nigra', 'Quercus laceyi', 'Quercus phellos', 'Quercus shumardii', 'Melia azedarach', 'Acer floridanum(Chapm.) Pax', 'Sabal palmetto(Walter) Lodd. ex Schult. & Schult. f.', 'Macromeria viridiflora', 'Gordonia lasianthus(L.) Ellis', 'Poa gracillima', 'Acacia saligna(Labill.) ', 'Fraxinus nigra', 'Dicentra pauciflora', 'Poa tracyi', 'Betula papyrifera', 'Besseya alpina', 'Tilia americana L.', 'Fraxinus L.', 'Ostrya virginiana', 'Solidago speciosa Nutt.', 'Vernonia baldwinii', 'Asclepias verticillata L.', 'Lemmonia californica', 'Teucrium laciniatum', 'Apocynum cannabinum', 'Baptisia australis', 'Sicyos laciniatus', 'Liatris mucronata', 'Solidago riddellii', 'Monarda fistulosa L.', 'Ratibida columnifera', 'Verbena stricta Vent.', 'Desmanthus illinoensis', 'Asclepias tuberosa L.', 'Echinacea pallida', 'Dichanthelium oligosanthes', 'Eryngium yuccifolium', 'Gleditsia triacanthos', 'Quercus ×macnabiana', 'Polycarpon depressum', 'Quercus muehlenbergii Engelm.', 'Andropogon gerardii Vitman', 'Trichomanes davallioides', 'Panicum virgatum', 'Schizachyrium scoparium(Michx.) Nash', 'Sophora nuttalliana', 'Helianthus angustifolius L.', 'Spartina pectinata', 'Lechea villosa', 'Amaranthus arenicola', 'Koeleria macrantha(Ledeb.) Schult.', 'Typha latifolia', 'Morus alba L.', 'Poa pratensis L.', 'Solanum campechiense', 'Helianthus dalyi', 'Ambrosia trifida', 'Rhexia aristosa Britton', 'Carex illota', 'Syringa vulgaris', 'Phacelia argentea', 'Carya ovalis', 'Cedronella canariensis', 'Helianthus sp.', 'Rhexia mariana L.', 'Kalmia latifolia L.', 'Malva sylvestris L.', 'Ilex montana', 'Rosa multiflora', 'Albizia julibrissin', 'Celtis laevigata', 'Distichlis spicata (L.) Greene', 'Ulmus alata', 'Rhus glabra', 'Cacalia decomposita', 'Osmunda cinnamomea L.', 'Magnolia acuminata(L.) L.', 'Acer pensylvanicum L.', 'Betula lenta', 'Delphinium purpusii', 'Hackelia virginiana', 'Proserpinaca pectinata', 'Palafoxia rosea(Bush) Cory', 'Madia madioides(Nutt.) Greene', 'Quercus ×palaeolithicola', 'Salix nigra', 'Bubblegum Plum', 'Panax quinquefolius L.', 'Oxydendrum arboreum(L.) DC.', 'Conyza floribunda', 'Arabis sparsiflora', 'Monarda russeliana', 'Quercus stellata', 'Prunus virginiana L.', 'Salix alaxensis', 'Nymphaea sagittifolia', 'Acacia baileyana', 'Calla palustris', 'Quercus lyrata', 'Acarospora nitida', 'Quercus acuminata(Michx.) Houba', 'Arundinaria gigantea', 'Nymphoides aquatica (J.F. Gmel.) Kuntze', 'Betula nigra L.', 'Juglans cinerea', 'Hackelia diffusa(Lehm.) I.M. Johnst.', 'Elaeagnus angustifolia', 'Salix L', 'Phacelia austromontana', 'Ulmus pumila L.', 'Poa alpigena', 'Malus spp.']
ds25_asd_english_name = ['Black walnut', 'Black oak', 'Cankerweed', 'Riverbank grape', 'American sycamore', 'Shaggy crabgrass', np.nan, 'Pawpaw', 'Tree of heaven', 'Tuliptree', 'Red maple', 'Dogwood', 'Rio Grande stickpea', 'New Mexico copperleaf', 'Northern red oak', 'White ash', 'Desertbeauty', 'Green ash', 'Torrid panicgrass', 'White oak', 'Mohr oak', "Dane's blood", 'Southern red oak', 'American beech', 'Tolmie star-tulip', 'Carolina fanwort', 'Ridged yellow flax', 'American elm', 'Blackgum', 'Black maple', 'Common hackberry', 'Black locust', 'Osage-orange', 'Maidenhair tree', 'Live oak', 'Bur oak', 'Bartram oak', 'Water oak', 'Lacey oak', 'Willow oak', "Shumard's oak", 'Chinaberrytree', 'Southern sugar maple', 'Cabbage palmetto', 'Giant-trumpets', 'Loblolly bay', 'Slender bluegrass', 'Orange wattle', 'Black ash', "Shorthorn steer's-head", "Tracy's bluegrass", 'Paper birch', 'Alpine besseya', 'American basswood', 'Ash', 'Hophornbeam', 'Solidago speciosa', "Baldwin's ironweed", 'Whorled milkweed', 'California fiddleleaf', 'Lacy germander', 'Indianhemp', 'Blue wild indigo', 'Cutleaf bur cucumber', 'narrow-leaved gayfeather', "Riddell's Goldenrod", 'Wild bergamot', 'Upright prairie coneflower', 'Hoary verbena', 'Illinois bundleflower', 'Butterfly milkweed', 'Pale purple coneflower', "Heller's rosette grass", 'Bbutton eryngo', 'Honeylocust', 'MacNab oak', 'California manyseed', 'Chinquapin oak', 'Big bluestem', 'Kilau', 'Switchgrass', 'Little bluestem', 'Silky sophora', 'Swamp sunflower', 'Prairie cordgrass', 'Hairy pinweed', 'Sandhill amaranth', 'Prairie Junegrass', 'Broadleaf cattail', 'White mulberry', 'Kentucky bluegrass', 'Redberry nightshade', 'Maximilian sunflower', 'Great ragweed', 'Awnpetal meadowbeauty', 'Sheep sedge', 'Common lilac', 'Sanddune phacelia', 'Red hickory', 'Herb of Gilead', 'Sunflower', 'Maryland meadowbeauty', 'Mountain laurel', 'High mallow', 'Mountain holly', 'Multiflora rose', 'Silktree', 'Sugarberry', 'Saltgrass', 'Winged elm', 'Smooth sumac', 'Psacalium decompositum', 'Cinnamon fern', 'Cucumber tree', 'Striped maple', 'Sweet birch', 'Kern County larkspur', 'Beggarslice', 'Combleaf mermaidweed', 'Rosy palafox', 'Woodland madia', 'Hybrid oak', 'Black willow', 'Bubblegum Plum', 'American ginseng', 'Sourwood', 'Asthmaweed', 'Sicklepod rockcress', 'Redpurple beebalm', 'Post oak', 'Chokecherry', 'Feltleaf willow', 'Yellow pond-lily', 'Cootamundra wattle', 'Water arum', 'Overcup oak', 'Cracked lichen', 'Mapleleaf oak', 'Giant cane', 'Big floatingheart', 'River birch', 'Butternut', 'Spreading stickseed', 'Russian olive', 'Willow', 'Southern Sierra phacelia', 'Siberian elm', 'Kentucky Bluegrass', 'Crabapple']
ds25_asd_pft = ['Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Vine', 'Deciduous broadleaf forests', 'Grasslands', np.nan, 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', np.nan, 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests']

for i in range(len(ds25_traits_asd['usda_code'].unique())):
    ds25_traits_asd.loc[ds25_traits_asd['usda_code'] == ds25_traits_asd['usda_code'].unique()[i], 'Latin Name'] = ds25_asd_latin_name[i]
    ds25_traits_asd.loc[ds25_traits_asd['usda_code'] == ds25_traits_asd['usda_code'].unique()[i], 'English Name'] = ds25_asd_english_name[i]
    ds25_traits_asd.loc[ds25_traits_asd['usda_code'] == ds25_traits_asd['usda_code'].unique()[i], 'PFT'] = ds25_asd_pft[i]

ds25_traits_asd.drop(['Project', 'PI','Affiliation','Functional_type'],axis = 1, inplace = True)
ds25_traits_asd['LMA'] = ds25_traits_asd['LMA']/10000

ds25_traits_asd["Dataset ID"] = 'Dataset#25'
k = 73
for i in ds25_traits_asd['Domain'].unique():
    ds25_traits_asd.loc[ds25_traits_asd['Domain']==i, "Site ID"] ='Site#'+str(k)
    k = k+1
    
ds25_traits_asd['Dataset Name'] = 'Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States'
ds25_traits_asd["Foreoptic type"] = "Contact probe"
ds25_traits_asd["Instruments"] = "ASD FieldSpec 3"
ds25_traits_asd["Reference"] = "(Wang, 2019)"
ds25_traits_asd["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-spectra-to-estimate-lma-over-neon-domains-in-eastern-united-states"
ds25_traits_asd = ds25_traits_asd[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA", "Foreoptic type", "Instruments",
                         "Reference","Link to original dataset"]]
###########
df14_refl_psr = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_psr_spectra.csv')
df14_refl_meta_psr = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_psr_metadata.csv')
df14_traits_psr = pd.read_csv('../1_datasets/0_original/25_Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States/neon2017_psr_lma.csv')

df14_all_psr = pd.merge(df14_refl_psr,df14_refl_meta_psr,how = 'left', on = 'Sample_ID')
df14_all_psr = pd.merge(df14_all_psr,df14_traits_psr,how = 'left', on = 'Sample_ID')
ds25_reflectance_psr = df14_all_psr.loc[:,'350':'2500']
ds25_traits_psr = df14_all_psr.loc[:,'USDA Symbol':]

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D02', 'Latitude'] = 38.8885357
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D02', 'Longitude'] = -78.1672628

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D03', 'Latitude'] = 29.6989925
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D03', 'Longitude'] = -82.3300268

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D05', 'Latitude'] = 46.2068814
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D05', 'Longitude'] = -89.4843009

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D06', 'Latitude'] = 39.1948221
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D06', 'Longitude'] = -96.5570739

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D07', 'Latitude'] = 36.0294079
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D07', 'Longitude'] = -84.2283144

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D08', 'Latitude'] = 33.1529079
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D08', 'Longitude'] = -87.5503075

ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D09', 'Latitude'] = 46.8936068
ds25_traits_psr.loc[ds25_traits_psr['Domain'] == 'D09', 'Longitude'] = -98.6879072

ds25_traits_psr.rename(columns={'USDA Symbol':'usda_code'},inplace = True)

ds25_psr_latin_name = ['Juglans nigra L.', 'Quercus velutina', 'Prenanthes serpentaria', 'Vitis riparia', 'Platanus occidentalis', 'Digitaria villosa', np.nan, 'Asimina triloba(L.) Dunal', 'Ailanthus altissima(Mill.) Swingle', 'Liriodendron tulipifera', 'Acer rubrum', 'Cornus sp.', 'Calliandra conferta', 'Acalypha neomexicana', 'Quercus rubra', 'Fraxinus americana', 'Linanthus bellus', 'Fraxinus pennsylvanica', 'Panicum torridum', 'Quercus alba', 'Quercus mohriana', 'Campanula glomerata', 'Quercus falcata', 'Fagus grandifolia', 'Calochortus tolmiei', 'Cabomba caroliniana', 'Linum striatum', 'Ulmus americana', 'Nyssa sylvatica', 'cer saccharum', 'Celtis occidentalis L.', 'Robinia pseudoacacia L.', 'Maclura pomifera', 'Ginkgo biloba L.', 'Quercus virginiana Mill.', 'Quercus macrocarpa', 'Quercus ×heterophylla', 'Quercus nigra', 'Quercus laceyi', 'Quercus phellos', 'Quercus shumardii', 'Melia azedarach', 'Acer floridanum(Chapm.) Pax', 'Sabal palmetto(Walter) Lodd. ex Schult. & Schult. f.', 'Macromeria viridiflora', 'Gordonia lasianthus(L.) Ellis', 'Poa gracillima', 'Acacia saligna(Labill.) ', 'Fraxinus nigra', 'Dicentra pauciflora', 'Poa tracyi', 'Betula papyrifera', 'Besseya alpina', 'Tilia americana L.', 'Fraxinus L.', 'Ostrya virginiana', 'Solidago speciosa Nutt.', 'Lechea villosa', 'Vernonia baldwinii', 'Asclepias verticillata L.', 'Lemmonia californica', 'Teucrium laciniatum', 'Apocynum cannabinum', 'Baptisia australis', 'Sicyos laciniatus', 'Liatris mucronata', 'Solidago riddellii', 'Monarda fistulosa L.', 'Ratibida columnifera', 'Verbena stricta Vent.', 'Desmanthus illinoensis', 'Asclepias tuberosa L.', 'Echinacea pallida', 'Dichanthelium oligosanthes', 'Eryngium yuccifolium', 'Gleditsia triacanthos', 'Quercus ×macnabiana', 'Polycarpon depressum', 'Andropogon gerardii Vitman', 'Trichomanes davallioides', 'Panicum virgatum', 'Schizachyrium scoparium(Michx.) Nash', 'Sophora nuttalliana', 'Helianthus angustifolius L.', 'Spartina pectinata', 'Amaranthus arenicola', 'Koeleria macrantha(Ledeb.) Schult.', 'Typha latifolia', 'Pectis tenella', 'Morus alba L.', 'Amaranthus californicus(Moq.) S. Watson', 'Poa pratensis L.', 'Solanum campechiense', 'Helianthus dalyi', 'Ambrosia trifida', 'Rhexia aristosa Britton', 'Quercus muehlenbergii Engelm.', 'Carex illota', 'Syringa vulgaris', 'Phacelia argentea', 'Carya ovalis', 'Cedronella canariensis', 'Helianthus sp.', 'Rhexia mariana L.', 'Kalmia latifolia L.', 'Malva sylvestris L.', 'Ilex montana', 'Rosa multiflora', 'Albizia julibrissin', 'Celtis laevigata', 'Distichlis spicata (L.) Greene', 'Ulmus alata', 'Rhus glabra', 'Cacalia decomposita', 'Osmunda cinnamomea L.', 'Magnolia acuminata(L.) L.', 'Acer pensylvanicum L.', 'Betula lenta', 'Delphinium purpusii', 'Hackelia virginiana', 'Proserpinaca pectinata', 'Palafoxia rosea(Bush) Cory', 'Madia madioides(Nutt.) Greene', 'Quercus ×palaeolithicola', 'Salix nigra', 'Bubblegum Plum', 'Panax quinquefolius L.', 'Oxydendrum arboreum(L.) DC.', 'Conyza floribunda', 'Arabis sparsiflora', 'Monarda russeliana', 'Quercus stellata', 'Prunus virginiana L.', 'Salix alaxensis', 'Nymphaea sagittifolia', 'Acacia baileyana', 'Calla palustris', 'Quercus lyrata', 'Acarospora nitida', 'Quercus acuminata(Michx.) Houba', 'Arundinaria gigantea', 'Nymphoides aquatica (J.F. Gmel.) Kuntze', 'Betula nigra L.', 'Juglans cinerea', 'Hackelia diffusa(Lehm.) I.M. Johnst.', 'Elaeagnus angustifolia', 'Salix L', 'Salix', 'Salix', 'Phacelia austromontana', 'Ulmus pumila L.', 'Poa alpigena', 'Malus spp.']
ds25_psr_english_name = ['Black walnut', 'Black oak', 'Cankerweed', 'Riverbank grape', 'American sycamore', 'Shaggy crabgrass', np.nan, 'Pawpaw', 'Tree of heaven', 'Tuliptree', 'Red maple', 'Dogwood', 'Rio Grande stickpea', 'New Mexico copperleaf', 'Northern red oak', 'White ash', 'Desertbeauty', 'Green ash', 'Torrid panicgrass', 'White oak', 'Mohr oak', "Dane's blood", 'Southern red oak', 'American beech', 'Tolmie star-tulip', 'Carolina fanwort', 'Ridged yellow flax', 'American elm', 'Blackgum', 'Black maple', 'Common hackberry', 'Black locust', 'Osage-orange', 'Maidenhair tree', 'Live oak', 'Bur oak', 'Bartram oak', 'Water oak', 'Lacey oak', 'Willow oak', "Shumard's oak", 'Chinaberrytree', 'Southern sugar maple', 'Cabbage palmetto', 'Giant-trumpets', 'Loblolly bay', 'Slender bluegrass', 'Orange wattle', 'Black ash', "Shorthorn steer's-head", "Tracy's bluegrass", 'Paper birch', 'Alpine besseya', 'American basswood', 'Ash', 'Hophornbeam', 'Solidago speciosa', 'Hairy pinweed', "Baldwin's ironweed", 'Whorled milkweed', 'California fiddleleaf', 'Lacy germander', 'Indianhemp', 'Blue wild indigo', 'Cutleaf bur cucumber', 'narrow-leaved gayfeather', "Riddell's Goldenrod", 'Wild bergamot', 'Upright prairie coneflower', 'Hoary verbena', 'Illinois bundleflower', 'Butterfly milkweed', 'Pale purple coneflower', "Heller's rosette grass", 'Bbutton eryngo', 'Honeylocust', 'MacNab oak', 'California manyseed', 'Big bluestem', 'Kilau', 'Switchgrass', 'Little bluestem', 'Silky sophora', 'Swamp sunflower', 'Prairie cordgrass', 'Sandhill amaranth', 'Prairie Junegrass', 'Broadleaf cattail', 'Mexican chinchweed', 'White mulberry', 'California amaranth', 'Kentucky bluegrass', 'Redberry nightshade', 'Maximilian sunflower', 'Great ragweed', 'Awnpetal meadowbeauty', 'Chinquapin oak', 'Sheep sedge', 'Common lilac', 'Sanddune phacelia', 'Red hickory', 'Herb of Gilead', 'Sunflower', 'Maryland meadowbeauty', 'Mountain laurel', 'High mallow', 'Mountain holly', 'Multiflora rose', 'Silktree', 'Sugarberry', 'Saltgrass', 'Winged elm', 'Smooth sumac', 'Psacalium decompositum', 'Cinnamon fern', 'Cucumber tree', 'Striped maple', 'Sweet birch', 'Kern County larkspur', 'Beggarslice', 'Combleaf mermaidweed', 'Rosy palafox', 'Woodland madia', 'Hybrid oak', 'Black willow', 'Bubblegum Plum', 'American ginseng', 'Sourwood', 'Asthmaweed', 'Sicklepod rockcress', 'Redpurple beebalm', 'Post oak', 'Chokecherry', 'Feltleaf willow', 'Yellow pond-lily', 'Cootamundra wattle', 'Water arum', 'Overcup oak', 'Cracked lichen', 'Mapleleaf oak', 'Giant cane', 'Big floatingheart', 'River birch', 'Butternut', 'Spreading stickseed', 'Russian olive', 'Willow', 'Willow', 'Willow', 'Southern Sierra phacelia', 'Siberian elm', 'Kentucky Bluegrass', 'Crabapple']
ds25_psr_pft = ['Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Vine', 'Deciduous broadleaf forests', 'Grasslands', np.nan, 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Shrublands', 'Grasslands', 'Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Shrublands', 'Grasslands', 'Deciduous broadleaf forests', np.nan, 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests']

for i in range(len(ds25_traits_psr['usda_code'].unique())):
    ds25_traits_psr.loc[ds25_traits_psr['usda_code'] == ds25_traits_psr['usda_code'].unique()[i], 'Latin Name'] = ds25_psr_latin_name[i]
    ds25_traits_psr.loc[ds25_traits_psr['usda_code'] == ds25_traits_psr['usda_code'].unique()[i], 'English Name'] = ds25_psr_english_name[i]
    ds25_traits_psr.loc[ds25_traits_psr['usda_code'] == ds25_traits_psr['usda_code'].unique()[i], 'PFT'] = ds25_psr_pft[i]

ds25_traits_psr.drop(['Project', 'PI','Functional_type','Affiliation'],axis = 1, inplace = True)
ds25_traits_psr['LMA'] = ds25_traits_psr['LMA']/10000
    
    
ds25_traits_psr["Dataset ID"] = 'Dataset#25'
k = 73
for i in ds25_traits_psr['Domain'].unique():
    ds25_traits_psr.loc[ds25_traits_psr['Domain']==i, "Site ID"] ='Site#'+str(k)
    k = k+1
    
ds25_traits_psr['Dataset Name'] = 'Fresh Leaf Spectra to Estimate LMA over NEON domains in eastern United States'
ds25_traits_psr["Foreoptic type"] = "Contact probe"
ds25_traits_psr["Instruments"] = "PSR+"
ds25_traits_psr["Reference"] = "(Wang, 2019)"
ds25_traits_psr["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-spectra-to-estimate-lma-over-neon-domains-in-eastern-united-states"

ds25_traits_psr = ds25_traits_psr[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT",
                         "Latitude","Longitude","LMA", "Foreoptic type", "Instruments",
                         "Reference","Link to original dataset"]]

ds25_traits = pd.concat([ds25_traits_asd,ds25_traits_psr],axis = 0)
ds25_reflectance = pd.concat([ds25_reflectance_asd,ds25_reflectance_psr],axis = 0)
ds25_traits.reset_index(drop = True, inplace = True)
ds25_reflectance.reset_index(drop = True, inplace = True)
ds25_traits["DOI"] = "https://doi.org/doi:10.21232/9831-rq60"

ds25_reflectance = ds25_reflectance.loc[:,"450":"2400"]
ds25_reflectance = do_resample(ds25_reflectance)
ds25 = pd.concat([ds25_reflectance, ds25_traits],axis = 1)
# ds25

### 26. 2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest

In [28]:
ds26_reflectance = pd.read_csv('../1_datasets/0_original/26_2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest/Brazil_ASD_Leaf_Spectra_filter_v1.csv')
ds26_reflectance = ds26_reflectance.T
ds26_reflectance.drop('Wavelength',inplace = True)
ds26_reflectance.columns = np.arange(350,2501).astype(str)
ds26_reflectance.reset_index(drop = True, inplace = True)


ds26_traits = pd.read_csv('../1_datasets/0_original/26_2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest/Brazil_Trait_Data_filter_v1.csv')
ds26_species = pd.read_csv('../1_datasets/0_original/26_2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest/Brazil_Species_Name.csv')
ds26_traits['Latin Name'] = ds26_species['Science Name']
ds26_traits['Sample date'] = ds26_species['Day']

ds26_traits.drop(['Leaf_Number','Quality_Control','Light_Environment','Leaf_Age','Complete_Leaf','Comments'],axis = 1, inplace = True)
ds26_traits.rename(columns={'SLA_m2_kg':'SLA','Water_Perc':'RWC'},inplace = True)

ds26_traits['Latitude'] = -4.0474179
ds26_traits['Longitude'] =  -54.9328055
ds26_traits['SLA'] = ds26_traits['SLA']*10
ds26_traits['LMA'] = 1/ds26_traits['SLA']

pfts = ['Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests',
        'Evergreen broadleaf forests','Evergreen broadleaf forests','Shrublands','Evergreen broadleaf forests',
        'Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests',np.nan,
        'Evergreen broadleaf forests','Evergreen broadleaf forests','Deciduous broadleaf forests',
        'Evergreen broadleaf forests','Evergreen broadleaf forests','Evergreen broadleaf forests']

for i in range(len(ds26_traits['Latin Name'].unique())):
    ds26_traits.loc[ds26_traits['Latin Name'] == ds26_traits['Latin Name'].unique()[i], 'PFT'] = pfts[i]\

ds26_traits["Dataset ID"] = 'Dataset#26'
ds26_traits["Site ID"] = 'Site#80'
ds26_traits['Dataset Name'] = '2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest'
ds26_traits["Foreoptic type"] = "Leaf clip"
ds26_traits["Instruments"] = "ASD FieldSpec Pro"
ds26_traits["Reference"] = "(Wu et al., 2017; Wu; et al., 2019)"
ds26_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/2012-leaf-reflectance-spectra-of-tropical-trees-in-tapajos-national-forest"
ds26_traits["DOI"] = "https://doi.org/10.1111/nph.14051"
ds26_traits = ds26_traits[["Dataset ID","Dataset Name","Site ID","Sample date","Latin Name","PFT",
                         "Latitude","Longitude","LMA", "Foreoptic type", "Instruments",
                         "Reference","Link to original dataset","DOI"]]

ds26_reflectance = ds26_reflectance.loc[:,"450":"2400"]
ds26_reflectance = do_resample(ds26_reflectance)
ds26 = pd.concat([ds26_reflectance, ds26_traits],axis = 1)
# ds26

### 27. 2014-leaf-reflectance-spectra-of-tropical-plants-growing-in-biosphere2

In [29]:
ds27_reflectance = pd.read_csv('../1_datasets/0_original/27_2014-leaf-reflectance-spectra-of-tropical-plants-growing-in-biosphere2/biosphere2_leaf_spectra_v1.csv')
ds27_reflectance.drop(['Leaf_ID'],axis = 1, inplace = True)

ds27_traits = pd.read_csv('../1_datasets/0_original/27_2014-leaf-reflectance-spectra-of-tropical-plants-growing-in-biosphere2/biosphere2_lma_v1.csv')

ds27_traits['Latitude'] = 32.5806964
ds27_traits['Longitude'] = -110.8553451
ds27_traits.drop(['Leaf_ID'],axis = 1, inplace = True)
ds27_traits.rename(columns={'LMA_gDW_m2':'LMA'},inplace = True)
ds27_traits['LMA'] = ds27_traits['LMA']/10000

ds27_traits['Sample date'] = '2014'
ds27_traits["Dataset ID"] = 'Dataset#27'
ds27_traits['Site ID'] = 'Site#81'
ds27_traits['Dataset Name'] = '2014-leaf-reflectance-spectra-of-tropical-plants-growing-in-biosphere2'
ds27_traits["Foreoptic type"] = "Leaf clip"
ds27_traits["Instruments"] = "ASD FieldSpec Pro"
ds27_traits["Reference"] = "(Wu et al., 2019)"
ds27_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/2014-leaf-reflectance-spectra-of-tropical-plants-growing-in-biosphere2"
ds27_traits["DOI"] = "https://ecosis.org/package/6a5ee8bb-87f5-4cf6-bb91-0b7602186347"
ds27_traits = ds27_traits[["Dataset ID","Dataset Name","Site ID","Sample date","Latitude","Longitude","LMA","Foreoptic type", "Instruments","Reference","Link to original dataset","DOI"]]

ds27_reflectance = ds27_reflectance.loc[:,"450":"2400"]
ds27_reflectance = do_resample(ds27_reflectance)
ds27 = pd.concat([ds27_reflectance, ds27_traits],axis = 1)
# ds27

### 28. Leaf spectra, structural and biochemical leaf traits of eight crop species

In [30]:
df15_refl = pd.read_csv('../1_datasets/0_original/28_Leaf spectra, structural and biochemical leaf traits of eight crop species/bnl_2015glasshouse_svc_averaged_interpolated_spectra.csv')
df15_refl_meta = pd.read_csv('../1_datasets/0_original/28_Leaf spectra, structural and biochemical leaf traits of eight crop species/bnl_2015glasshouse_metadata.csv')
df15_traits = pd.read_csv('../1_datasets/0_original/28_Leaf spectra, structural and biochemical leaf traits of eight crop species/bnl_2015glasshouse_leaf_traits.csv')

df15_all = pd.merge(df15_refl,df15_refl_meta,how = 'left', on = 'sample_id')
df15_all = pd.merge(df15_all,df15_traits,how = 'left', on = 'sample_id')

ds28_reflectance = df15_all.loc[:,'350':'2500']
ds28_reflectance = ds28_reflectance/100.0

ds28_traits = df15_all.loc[:,'species':]

ds28_traits['Latitude'] = 40.8640186
ds28_traits['Longitude'] = -72.8694428

ds28_traits.rename(columns={'species':'usda_code','C_pc_dry':'Cmass(%)','H_pc_dry':'Hydrogen(%)',
                            'N_pc_dry': 'Nmass(%)','LMA_g_m2': 'LMA','H20_g_m2':'EWT','C_g_m2':'Carea',
                            'N_g_m2': 'Narea','H2O_pc': 'RWC'},inplace = True)

ds28_latin_name =['Helianthus annuus L.', 'Cucumis sativus L.', 'Cucurbita pepo L.', 'Solanum lycopersicum L.', 'Ocimum basilicum L.', 'Populus L.', 'Glycine max', 'Phaseolus vulgaris L.']
ds28_english_name = ['Common sunflower', 'Garden cucumber', 'Field pumpkin', 'Garden tomato', 'Sweet basil', 'Cottonwood', 'Soybean', 'Kidney bean']
ds28_pft = ['Croplands', 'Croplands', 'Croplands', 'Croplands', 'Croplands', 'Croplands', 'Croplands', 'Croplands']

for i in range(len(ds28_traits['usda_code'].unique())):
    ds28_traits.loc[ds28_traits['usda_code'] == ds28_traits['usda_code'].unique()[i], 'Latin Name'] = ds28_latin_name[i]
    ds28_traits.loc[ds28_traits['usda_code'] == ds28_traits['usda_code'].unique()[i], 'English Name'] = ds28_english_name[i]
    ds28_traits.loc[ds28_traits['usda_code'] == ds28_traits['usda_code'].unique()[i], 'PFT'] = ds28_pft[i]

ds28_traits['LMA'] = ds28_traits['LMA']/10000
ds28_traits['EWT'] = ds28_traits['EWT']/10000

ds28_traits["Dataset ID"] = 'Dataset#28'
ds28_traits['Site ID'] = 'Site#82'
ds28_traits['Dataset Name'] = 'Leaf spectra, structural and biochemical leaf traits of eight crop species'
ds28_traits["Foreoptic type"] = "Leaf clip"
ds28_traits["Instruments"] = "SVC HR-1024i"
ds28_traits["Reference"] = "(Ely et al., 2018, 2019)"
ds28_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-spectra--structural-and-biochemical-leaf-traits-of-eight-crop-species"
ds28_traits["DOI"] = "https://doi.org/doi:10.21232/C2GM2Z"
ds28_traits = ds28_traits[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT","Latitude","Longitude",
                           "EWT","LMA","Foreoptic type", "Instruments","Reference","Link to original dataset","DOI"]]

ds28_reflectance = ds28_reflectance.loc[:,"450":"2400"]
ds28_reflectance = do_resample(ds28_reflectance)
ds28 = pd.concat([ds28_reflectance, ds28_traits],axis = 1)
# ds28

### 29. Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests

In [31]:
df16_refl = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftdryspectrav2.csv')
df16_refl_meta = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftdrymetadata.csv')

df16_traits1 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftlma.csv')
df16_traits2 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftnitrogen.csv')
df16_traits3 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftcellulose.csv')
df16_traits4 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftcarbon.csv')
df16_traits5 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftadl.csv')
df16_traits6 = pd.read_csv('../1_datasets/0_original/29_Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests/fftadf.csv')

df16_refl_merge = pd.merge(df16_refl,df16_refl_meta,how = 'left', on = 'UniqueID')

df16_all = pd.merge(df16_refl_merge,df16_traits1,how = 'left', on = 'Sample_Name')
df16_all = pd.merge(df16_all,df16_traits2,how = 'left', on = 'Sample_Name')
df16_all = pd.merge(df16_all,df16_traits3,how = 'left', on = 'Sample_Name')
df16_all = pd.merge(df16_all,df16_traits4,how = 'left', on = 'Sample_Name')
df16_all = pd.merge(df16_all,df16_traits5,how = 'left', on = 'Sample_Name')
df16_all = pd.merge(df16_all,df16_traits6,how = 'left', on = 'Sample_Name')

df16_all = df16_all[(df16_all['LMA_gm2'].notnull())|(df16_all['nitrogen_percent'].notnull())|(df16_all['Cellulose_percent'].notnull())|(df16_all['Carbon_percent'].notnull())|(df16_all['Lignin_percent'].notnull())|(df16_all['Fiber_percent'].notnull())]
df16_all.reset_index(drop = True, inplace = True)

ds29_reflectance = df16_all.loc[:,'350':'2500']
ds29_traits = df16_all.loc[:,'Site':]

sites = ['Adirondack Park,NY','Baraboo Hills,WI','Blackhawk Island,WI','Madison,WI','Kettle Moraine State Forest',
         'Finland State Forest, Crystal Bay Township','Flambeau State Forest,WI','Ottawa National Forest',
         'Moquah Barrens State Natural Area','Porcupine Mountains,MI','Superior National Forest']

for i in range(len(sites)):
    geolocator = Nominatim(user_agent='BuyiXiao',timeout=5)
    location = geolocator.geocode(sites[i])
    if location is None:
        pass
    else:
        ds29_traits.loc[ds29_traits['Site_Name'] == ds29_traits['Site_Name'].unique()[i], 'Latitude'] = location.latitude
        ds29_traits.loc[ds29_traits['Site_Name'] == ds29_traits['Site_Name'].unique()[i], 'Longitude'] = location.longitude

ds29_traits.loc[ds29_traits['Site_Name'] == 'Finland SF / Superior NF', 'latitude'] = 47.4521338
ds29_traits.loc[ds29_traits['Site_Name'] == 'Finland SF / Superior NF', 'longitude'] = -91.2246028

ds29_traits.rename(columns={'Species':'usda_code','LMA_gm2': 'LMA','nitrogen_percent':'Nmass(%)',
                            'Cellulose_percent':'Cellulose(%)','Carbon_percent':'Cmass(%)',
                            'Lignin_percent':'Lignin(%)','Fiber_percent':'Fiber(%)'},inplace = True)

ds29_latin_name = ['Picea rubens Sarg.', 'Besseya alpina', 'Acacia smallii', 'Fagus grandifolia', 'Acer rubrum', 'Betula papyrifera', 'Pinus strobus', 'Quercus alba', 'Quercus rubra', 'Tsuga canadensis(L.) Carrière', 'Carya ovalis', 'Fraxinus americana', 'Tilia americana L.', 'Prenanthes serpentaria', 'GRASS', 'Ostrya virginiana', 'Poa tracyi', 'Polycarpon depressum', 'Ulmus americana', 'Pinus resinosa', 'Fraxinus pennsylvanica', 'Arnica snyderi', 'Juglans nigra L.', 'Celtis occidentalis L.', 'Robinia pseudoacacia L.', 'Rhamnus californica', 'Quercus ×macnabiana', 'Liriodendron tulipifera', 'Acamptopappus sphaerocephalus', 'Conopholis americana (L.)', 'Abies balsamea(L.) Mill.', 'Pinus balfouriana', 'Betula glandulosa', 'Ledum groenlandicum', 'Picea mariana(Mill.) Britton, Sterns & Poggenb.', 'Allionia incarnata', 'Fraxinus nigra', 'Thalictrum occidentale', 'Larix laricina(Du Roi) K. Koch', 'Quercus ellipsoidalis']
ds29_english_name = ['Red spruce', 'Alpine besseya', 'Sweet acacia', 'American beech', 'Red maple', 'Paper birch', 'Eastern white pine', 'White oak', 'Northern red oak', 'Eastern hemlock', 'Red hickory', 'White ash', 'American basswood', 'Cankerweed', 'GRASS', 'Hophornbeam', "Tracy's bluegrass", 'California manyseed', 'American elm', 'Red pine', 'Green ash', 'Snow arnica', 'Black walnut', 'Common hackberry', 'Black locust', 'California buckthorn', 'MacNab oak', 'Tuliptree', 'Rayless goldenhead', 'American cancer-root', 'Balsam fir', 'Foxtail pine', 'Resin birch', 'Bog Labrador tea', 'Black spruce', 'Trailing windmills', 'Black ash', 'Western meadow-rue', 'Tamarack', 'Northern pin oak']
ds29_pft = ['Evergreen needleleaf forests', 'Grasslands', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Evergreen needleleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Grasslands', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands', 'Grasslands', 'Evergreen needleleaf forests', 'Deciduous needleleaf forests', 'Shrublands', 'Shrublands', 'Evergreen needleleaf forests', 'Grasslands', 'Deciduous broadleaf forests', 'Grasslands', 'Deciduous needleleaf forests', 'Deciduous broadleaf forests']

for i in range(len(ds29_traits['usda_code'].unique())):
    ds29_traits.loc[ds29_traits['usda_code'] == ds29_traits['usda_code'].unique()[i], 'Latin Name'] = ds29_latin_name[i]
    ds29_traits.loc[ds29_traits['usda_code'] == ds29_traits['usda_code'].unique()[i], 'English Name'] = ds29_english_name[i]
    ds29_traits.loc[ds29_traits['usda_code'] == ds29_traits['usda_code'].unique()[i], 'PFT'] = ds29_pft[i]
    
ds29_traits.drop(['Site','Plot','Height','Age','Height_Age','Measurement_Type','Measurement','Sample_Type',
                  'Instrumentation','Calibration','Wavelengths','Column_Units','Project','PI','Affiliation'],axis = 1, inplace = True)
ds29_traits['LMA'] = ds29_traits['LMA']/10000
ds29_traits['Dataset ID'] = 'Dataset#29'

j = 0
s = ['Site#54','Site#55','Site#42','Site#33','Site#58','Site#59','Site#60','Site#61','Site#62','Site#63','Site#64']
for i in ds29_traits['Site_Name'].unique():
    ds29_traits.loc[ds29_traits['Site_Name'] == i, 'Site ID'] = s[j]
    j = j+1

ds29_traits['Dataset Name'] = 'Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests'
ds29_traits["Foreoptic type"] = "Contact probe"
ds29_traits["Instruments"] = "ASD FieldSpec 3"
ds29_traits["Reference"] = "(Serbin et al., 2014; Serbin, 2016)"
ds29_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/dried-leaf-spectra-to-estimate-leaf-morphology-and-biochemistry-for-northern-temperate-forests"
ds29_traits["DOI"] = "https://doi.org/doi:10.21232/C21593"
ds29_traits = ds29_traits[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT","Latitude",
                           "Longitude", "LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds29_reflectance = ds29_reflectance.loc[:,"450":"2400"]
ds29_reflectance = do_resample(ds29_reflectance)
ds29 = pd.concat([ds29_reflectance, ds29_traits],axis = 1)
# ds29

### 30. Productivity and Characterization of Soybean Foliar Traits Under Aphid Pressure

In [32]:
df17_refl = pd.read_csv('../1_datasets/0_original/30_Productivity and Characterization of Soybean Foliar Traits Under Aphid Pressure/uwgreenhousesoyspectranowave-2.csv')
df17_refl_meta = pd.read_csv('../1_datasets/0_original/30_Productivity and Characterization of Soybean Foliar Traits Under Aphid Pressure/uwgreenhousesoyancillary-2.csv')
df17_traits = pd.read_csv('../1_datasets/0_original/30_Productivity and Characterization of Soybean Foliar Traits Under Aphid Pressure/uwgreenhousesoytraits-2.csv')

df17_all = pd.merge(df17_refl,df17_refl_meta,how = 'left', on = 'SAMP_ID')
df17_all = pd.merge(df17_all,df17_traits,how = 'left', on = 'SAMP_ID')

ds30_reflectance = df17_all.loc[:,'350':'2500']
ds30_traits = df17_all.loc[:,'SPECIES':]

ds30_traits.rename(columns={'LATITUDE':'Latitude','LONGITUDE':'Longitude','SPECIES':'Latin Name',
                            'CHL_a':'Chla','gmm2_LMA':'LMA','CHL_b':'Chlb','CAROTENOIDS':'Ccar',
                            'DATE':'Sample date','STATE':'State'},inplace = True)
ds30_traits['English Name'] = 'Soybean'
ds30_traits['PFT'] = 'Croplands'

ds30_traits.drop(['DATAFROM','INSTRUMENT','CITY','MM','DD','YYYY','STAGE','REP','PLANT','LEAF_HEIGHT'],axis = 1, inplace = True)
ds30_traits['Chla+b'] = ds30_traits['Chla'] + ds30_traits['Chlb']
ds30_traits['LMA'] = ds30_traits['LMA']/10000
ds30_traits.loc[ds30_traits['LMA']<0,'LMA'] = np.nan

sample_date = ['20130703','20130708','20130710','20130711','20130715','20130717','20130719','20130722','20130724','20130726','20130729', '20130731', '20130802','20130805','20130807','20130809','20130812','20130814','20130816','20130819','20130821','20130826']
for i in range(len(ds30_traits['Sample date'].unique())):
    ds30_traits.loc[ds30_traits['Sample date'] == ds30_traits['Sample date'].unique()[i], 'Sample date'] = sample_date[i]

ds30_traits["Dataset ID"] = 'Dataset#30'
ds30_traits["Site ID"] = 'Site#83'
ds30_traits['Dataset Name'] = 'Dried Leaf Spectra to Estimate Leaf Morphology and Biochemistry for Northern Temperate Forests'
ds30_traits["Foreoptic type"] = "Leaf clip"
ds30_traits["Instruments"] = "ASD FieldSpec 4"
ds30_traits["Reference"] = "(Singh, 2013)"
ds30_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/productivity-and-characterization-of-soybean-foliar-traits-under-aphid-pressure"
ds30_traits["DOI"] = "https://ecosis.org/package/cdbb6b09-b481-4022-a0da-ad95a8b085d8"
ds30_traits = ds30_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name",
                           "PFT","Latitude","Longitude","Chla+b","Ccar","LMA","Foreoptic type", "Instruments",
                           "Reference","Link to original dataset","DOI"]]

ds30_reflectance = ds30_reflectance.loc[:,"450":"2400"]
ds30_reflectance = do_resample(ds30_reflectance)
ds30 = pd.concat([ds30_reflectance, ds30_traits],axis = 1)
# ds30

### 31. Common Milkweed Leaf Responses to Water Stress and Elevated Temperature

In [33]:
df18_refl = pd.read_csv('../1_datasets/0_original/31_Common Milkweed Leaf Responses to Water Stress and Elevated Temperature/couture-et-al.-2015apisspectra.csv')
df18_refl_meta = pd.read_csv('../1_datasets/0_original/31_Common Milkweed Leaf Responses to Water Stress and Elevated Temperature/couture-et-al.-2015apismetadata.csv')
df18_traits = pd.read_csv('../1_datasets/0_original/31_Common Milkweed Leaf Responses to Water Stress and Elevated Temperature/couture-et-al.-2015apistraits.csv')

df18_all = pd.merge(df18_refl,df18_refl_meta,how = 'left', on = 'ID')
df18_all = pd.merge(df18_all,df18_traits,how = 'left', on = 'ID')

ds31_reflectance = df18_all.loc[:,'350':'2500']
ds31_traits = df18_all.loc[:,'Water treatment':]

ds31_traits['Latitude'] = 43.0760575
ds31_traits['Longitude'] = -89.4225098

ds31_traits['Latin Name'] = 'Asclepias syriaca'
ds31_traits['English Name'] = 'Common Milkweed'
ds31_traits['PFT'] = 'Grasslands'

ds31_traits['LMA'] = ds31_traits['LMA']/10000
ds31_traits['Dataset ID'] = 'Dataset#31'
ds31_traits['Site ID'] = 'Site#84'

ds31_traits['Dataset Name'] = 'Common Milkweed Leaf Responses to Water Stress and Elevated Temperature'
ds31_traits["Foreoptic type"] = "Leaf clip, Contact probe"
ds31_traits["Instruments"] = "ASD FieldSpec 3"
ds31_traits["Reference"] = "(Couture, 2015; Couture et al., 2015)"
ds31_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/common-milkweed-leaf-responses-to-water-stress-and-elevated-temperature"
ds31_traits["DOI"] = "https://doi.org/10.1007/s11829-015-9367-y"
ds31_traits = ds31_traits[["Dataset ID","Dataset Name","Site ID","English Name","Latin Name","PFT","Latitude",
                           "Longitude","LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds31_reflectance = ds31_reflectance.loc[:,"450":"2400"]
ds31_reflectance = do_resample(ds31_reflectance)
ds31 = pd.concat([ds31_reflectance, ds31_traits],axis = 1)
# ds31

### 32. Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops

In [34]:
lmabean_2014 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2014lmabean.csv')
lmacorn_2014 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2014lmacorn.csv')
refnbean_2014 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2014refnbean.csv')
refncorn_2014 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2014refncorn.csv')

lmabean_2015 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2015lmabean.csv')
lmacorn_2015 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2015lmacorn.csv')
refnbean_2015 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2015refnbean.csv')
refncorn_2015 = pd.read_csv('../1_datasets/0_original/32_Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops/2015refncorn.csv')

lmabean_2014['Latitude'] = 44.1197052
lmabean_2014['Longitude'] = -89.5353393

lmacorn_2014['Latitude'] = 44.1197052
lmacorn_2014['Longitude'] = -89.5353393

refnbean_2014['Latitude'] = 44.1197052
refnbean_2014['Longitude'] = -89.5353393

refncorn_2014['Latitude'] = 44.1197052
refncorn_2014['Longitude'] = -89.5353393

lmabean_2015['Latitude'] = 44.1197052
lmabean_2015['Longitude'] = -89.5353393

lmacorn_2015['Latitude'] = 44.1197052
lmacorn_2015['Longitude'] = -89.5353393

refnbean_2015['Latitude'] = 44.1197052
refnbean_2015['Longitude'] = -89.5353393

refncorn_2015['Latitude'] = 44.1197052
refncorn_2015['Longitude'] = -89.5353393

ds32_traits = pd.concat([lmabean_2014,lmabean_2015,lmacorn_2014,lmacorn_2015])
dates = ['2014-07-21','2014-07-28','2014-08-04','2014-08-13','2015-07-23','2015-07-29','2008-06-15','2015-08-18','2014-08-21']

for i in range(len(ds32_traits['date'].unique())):
    ds32_traits.loc[ds32_traits['date'] == ds32_traits['date'].unique()[i], 'date'] = dates[i]

ds32_reflectance = ds32_traits.loc[:,'350':'2500']
ds32_reflectance.columns = np.arange(350,2501).astype(int).astype(str)

ds32_traits = ds32_traits[['date','site','English Name', 'refLMA','Latitude','Longitude']]
ds32_traits.rename(columns={'refLMA':'LMA','date':'Sample date'},inplace = True)

ds32_traits['PFT'] = 'Croplands'
ds32_traits['LMA'] = ds32_traits['LMA']/10000
sample_date = ['20140721','20140728','20140804','20140813','20150723','20150729','20080615','20150818','20140821']

for i in range(len(ds32_traits['Sample date'].unique())):
    ds32_traits.loc[ds32_traits['Sample date'] == ds32_traits['Sample date'].unique()[i], 'Sample date'] = sample_date[i]

ds32_traits['Dataset ID'] = 'Dataset#32'
ds32_traits['Site ID'] = 'Site#85'

ds32_traits['Dataset Name'] = 'Fresh leaf spectra to estimate leaf N concentration and leaf mass per area in two vegetable crops'
ds32_traits["Foreoptic type"] = "Leaf clip"
ds32_traits["Instruments"] = "SVC HR-1024i"
ds32_traits["Reference"] = "(Eosis, 2016)"
ds32_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-spectra-to-estimate-leaf-n-concentration-and-leaf-mass-per-area-in-two-vegetable-crops"
ds32_traits["DOI"] = "https://ecosis.org/package/5b6b29b8-2250-4c24-a3ff-74366acca75a"
ds32_traits = ds32_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","PFT","Latitude",
                           "Longitude","LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds32_traits.reset_index(drop = True, inplace = True)
ds32_reflectance = ds32_reflectance.loc[:,"450":"2400"]
ds32_reflectance = do_resample(ds32_reflectance)
ds32 = pd.concat([ds32_reflectance, ds32_traits],axis = 1)
# ds32

### 33. Fresh Leaf VSWIR Spectra to Estimate Leaf Traits for California Ecosystems

In [35]:
ds33_reflectance = pd.read_csv('../1_datasets/0_original/33_Fresh Leaf VSWIR Spectra to Estimate Leaf Traits for California Ecosystems/spectral.csv')
ds33_reflectance.drop(['spectra'],axis = 1, inplace = True)

ds33_traits = pd.read_csv('../1_datasets/0_original/33_Fresh Leaf VSWIR Spectra to Estimate Leaf Traits for California Ecosystems/leaf_traits.csv')

ds33_traits.rename(columns={'species':'usda_code','Nitrogen':'Nmass(%)','Cellulose':'Cellulose(%)',
                            'Lignin':'Lignin(%)','Water Content':'RWC','Sample Date':'Sample date'},inplace = True)

ds33_latin_name = ['Baccharis pilularis', 'Ceanothus spinosus', 'Hedyosmum arborescens', 'Ceanothus megacarpus', 'Umbellularia californica', 'Ceanothus cuneatus', 'Adenostoma fasciculatum', 'Arabis glabra', 'Cacalia decomposita', 'Pinus lambertiana', 'Abies concolor', 'Pinus ponderosa', 'Quercus douglasii', 'Quercus agrifolia', 'Quercus lobata', 'Salix lemmonii']
ds33_english_name = ['Coyotebrush', 'Redheart', 'Cigarbush', 'Bigpod ceanothus', 'California laurel', 'Buckbrush', 'Chamise', 'Tower Rockcress', 'Psacalium decompositum', 'Sugar pine', 'White fir', 'Ponderosa pine', 'Blue oak', 'California live oak', 'Valley oak', "Lemmon's willow"]
ds33_pft = ['Shrublands', 'Shrublands', 'Deciduous broadleaf forests', 'Shrublands', 'Evergreen broadleaf forests', 'Shrublands', 'Shrublands', 'Grasslands', 'Grasslands', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Evergreen needleleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Deciduous broadleaf forests', 'Shrublands']

for i in range(len(ds33_traits['usda_code'].unique())):
    ds33_traits.loc[ds33_traits['usda_code'] == ds33_traits['usda_code'].unique()[i], 'Latin Name'] = ds33_latin_name[i]
    ds33_traits.loc[ds33_traits['usda_code'] == ds33_traits['usda_code'].unique()[i], 'English Name'] = ds33_english_name[i]
    ds33_traits.loc[ds33_traits['usda_code'] == ds33_traits['usda_code'].unique()[i], 'PFT'] = ds33_pft[i]
    
ds33_traits['LMA'] = ds33_traits['LMA']/10000
sample_date = ['20130401','20130420','20130421','20130603','20130608','20130609','20131013','20131102']

for i in range(len(ds33_traits['Sample date'].unique())):
    ds33_traits.loc[ds33_traits['Sample date'] == ds33_traits['Sample date'].unique()[i], 'Sample date'] = sample_date[i]

ds33_traits['Dataset ID'] = 'Dataset#33'
k = 86
for i in ds33_traits['site name'].unique():
    ds33_traits.loc[ds33_traits['site name']==i,'Site ID'] = 'Site#'+str(k)
    k = k+1

ds33_traits['Dataset Name'] = 'Fresh Leaf VSWIR Spectra to Estimate Leaf Traits for California Ecosystems'
ds33_traits["Foreoptic type"] = "Bare fiber"
ds33_traits["Instruments"] = "ASD FieldSpec 3"
ds33_traits["Reference"] = "(Meerdink, 2016)"
ds33_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/fresh-leaf-vswir-spectra-to-estimate-leaf-traits-for-california-ecosystems"
ds33_traits["DOI"] = "https://ecosis.org/package/0fadcc45-f79e-4fd3-a6ca-8afaf26ae299"
ds33_traits = ds33_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name",
                           "PFT","Latitude","Longitude","LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds33_reflectance = ds33_reflectance.loc[:,"450":"2400"]
ds33_reflectance = do_resample(ds33_reflectance)
ds33 = pd.concat([ds33_reflectance, ds33_traits],axis = 1)
# ds33

### 34. 2014 Cedar Creek ESR Grassland Biodiversity Experiment_Leaf-level Contact Data_Trait Predictions

In [36]:
ds34_reflectance = pd.read_csv('../1_datasets/0_original/34_2014 Cedar Creek ESR Grassland Biodiversity Experiment_Leaf-level Contact Data_Trait Predictions/cedarcreekleafspectra2014.csv')
ds34_reflectance.drop(['guid'],axis = 1, inplace = True)

ds34_traits = pd.read_csv('../1_datasets/0_original/34_2014 Cedar Creek ESR Grassland Biodiversity Experiment_Leaf-level Contact Data_Trait Predictions/cedarcreekmetadata2014.csv')
ds34_traits['Latitude'] = 45.4019246
ds34_traits['Longitude'] = -93.2003527
ds34_traits['PFT'] = 'Grasslands'


ds34_traits.rename(columns={'Chl g m2':'Chla+b','LMA g m2':'LMA'},inplace = True)

ds34_traits['Chla+b'] = ds34_traits['Chla+b']*100
ds34_traits['LMA'] = ds34_traits['LMA']/10000

ds34_traits.loc[ds34_traits['Sample Collection Date'] =='June', 'Sample date'] = '201406'
ds34_traits.loc[ds34_traits['Sample Collection Date'] =='July', 'Sample date'] = '201407'
ds34_traits.loc[ds34_traits['Sample Collection Date'] =='Sept', 'Sample date'] = '201409'


ds34_traits['Dataset ID'] = 'Dataset#34'
ds34_traits['Site ID'] = 'Site#5'

ds34_traits['Dataset Name'] = '2014 Cedar Creek ESR Grassland Biodiversity Experiment_Leaf-level Contact Data_Trait Predictions'
ds34_traits["Foreoptic type"] = "Leaf clip"
ds34_traits["Instruments"] = "ASD FieldSpec 3"
ds34_traits["Reference"] = "(Couture, 2016)"
ds34_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/2014-cedar-creek-esr-grassland-biodiversity-experiment--leaf-level-contact-data--trait-predictions"
ds34_traits["DOI"] = "https://ecosis.org/package/acbe2150-2fa7-467c-bd56-889ba7284d00"
ds34_traits = ds34_traits[["Dataset ID","Dataset Name","Site ID","Sample date","PFT","Latitude","Longitude",
                           "Chla+b","LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds34_reflectance = ds34_reflectance.loc[:,"450":"2400"]
ds34_reflectance = do_resample(ds34_reflectance)
ds34 = pd.concat([ds34_reflectance, ds34_traits],axis = 1)
# ds34

### 35. Leaf traits of temperate broadleaf deciduous trees

In [37]:
ds35_reflectance = pd.read_csv('../1_datasets/0_original/35_Predicting leaf traits of temperate broadleaf deciduous trees from hyperspectral reflectance_can a general model be applied across a growing season/Raw spectral data.csv')
ds35_traits = pd.read_csv('../1_datasets/0_original/35_Predicting leaf traits of temperate broadleaf deciduous trees from hyperspectral reflectance_can a general model be applied across a growing season/Raw trait data.csv',encoding= 'unicode_escape')

code = ds35_reflectance['Code_Spec'].unique()
r = ds35_reflectance[ds35_reflectance['Code_Spec'] == code[0]]
refl = r.groupby('Set').agg('mean').reset_index()
t = ds35_traits[ds35_traits['Code_Traits'] == code[0]]
t.reset_index(drop = True, inplace = True)
df_merge = pd.merge(t,refl,how = 'left', on = 'Set')

for i in code[1:]:
    rr = ds35_reflectance[ds35_reflectance['Code_Spec'] == i]
    refl1 = rr.groupby('Set').agg('mean').reset_index()
    tt = ds35_traits[ds35_traits['Code_Traits'] == i]
    tt.reset_index(drop = True, inplace = True)
    temp = pd.merge(tt,refl1,how = 'left', on = 'Set')
    df_merge = pd.concat([df_merge,temp])
    
ds35_reflectance = df_merge.loc[:,'350':]
ds35_reflectance.reset_index(drop = True, inplace = True)
ds35_traits = df_merge.loc[:,:'Boron (B, ¦Ìg/g)']
ds35_traits.reset_index(drop = True, inplace = True)

ds35_traits.drop(['Code_Traits','Set','Date','¦Ä15N (¡ë)','¦Ä13C (¡ë)','Iron (Fe, ¦Ìg/g)','Manganese (Mn, ¦Ìg/g)','Zinc (Zn, ¦Ìg/g)','Boron (B, ¦Ìg/g)','Tannin (mg/g)'],axis = 1, inplace = True)
ds35_traits.rename(columns={'Species':'English Name','Time':'Sample date','Nitrogen (N, %)':'Nmass(%)','LMA (g/m2)':'LMA',
                            'Total carbon (%)':'Cmass(%)','Soluble carbon (%)':'Solubles(%)','LWC':'RWC',
                            'Hemicellulose (%)':'Hemicellulose(%)','Cellulose (%)':'Cellulose(%)',
                            'Lignin (%)':'Lignin(%)','Total phenolics (mg/g)':'Phenolics_mass',
                            'Phosphorus (P, %)':'Pmass(%)','Potassium (K, %)':'Kmass(%)','Chlorophyll':'Chla+b',
                            'Calcium (Ca, %)':'Camass(%)','Magnesium (Mg, %)':'Mgmass(%)'},inplace = True)

latin_name = ['Fraxinus excelsior','Betula pubescens','Ulmus glabra','Acer campestre','Crataegus monogyna',
              'Corylus avellana','Quercus robur','Acer pseudoplatanus']

for i in range(len(ds35_traits['English Name'].unique())):
    ds35_traits.loc[ds35_traits['English Name'] == ds35_traits['English Name'].unique()[i], 'Latin Name'] = latin_name[i]

ds35_traits['LMA'] = ds35_traits['LMA']/10000
ds35_traits['Latitude'] = 52.2172964
ds35_traits['Longitude'] = 0.0486343
ds35_traits['PFT'] = 'Deciduous broadleaf forests'
ds35_traits['Dataset ID'] = 'Dataset#35'
ds35_traits['Site ID'] = 'Site#89'

ds35_traits['Dataset Name'] = 'Leaf traits of temperate broadleaf deciduous trees'
ds35_traits["Foreoptic type"] = "Contact probe"
ds35_traits["Instruments"] = "ASD FieldSpec 3"
ds35_traits["Reference"] = "(Chen et al., 2022)"
ds35_traits["Link to original dataset"] = "https://figshare.com/articles/dataset/Raw_leaf_trait_and_spectral_data/16909330/1"
ds35_traits["DOI"] = "https://doi.org/10.1016/j.rse.2021.112767 "
ds35_traits = ds35_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name","Latin Name",
                           "PFT","Latitude","Longitude","Chla+b","LMA","Foreoptic type", "Instruments",
                           "Reference","Link to original dataset","DOI"]]

ds35_reflectance = ds35_reflectance.loc[:,"450":"2400"]
ds35_reflectance = do_resample(ds35_reflectance)
ds35 = pd.concat([ds35_reflectance, ds35_traits],axis = 1)
# ds35

### 36. leaf traits and leaf spectroscopy of east Asian forest tree species

In [38]:
ds36_all = pd.read_csv('../1_datasets/0_original/36_Estimation of six leaf traits of East Asian forest tree species by leaf spectroscopy and partial least square regression/Leaf_traits-spectral_dataset.csv',encoding= 'unicode_escape')
ds36_reflectance = ds36_all.loc[:,'400':]
# ds36_reflectance = data_interpolate(ds36_reflectance)

ds36_traits = ds36_all.loc[:,:'Lignin(mg/g)']
ds36_traits.rename(columns={'Site':'site name','Sampling date':'Sample date','Species name':'Latin Name',
                            'LMA(g/m2)':'LMA'},inplace = True)

lat = [44.81,42.68,41.75,42.71,38.76,35.74,35.31,33.66,32.38,31.87,30.32,14.51,2.97,4.20]
lon = [142.08,140.26,140.15,141.57,140.75,139.54,133.60,135.67,131.16,131.30,130.64,101.94,102.30,114.04]
pfts = ['Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen needleleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Deciduous broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests', 'Evergreen broadleaf forests']

k = 90
for i in range(len(ds36_traits['site name'].unique())):
    ds36_traits.loc[ds36_traits['site name'] == ds36_traits['site name'].unique()[i], 'Latitude'] = lat[i]
    ds36_traits.loc[ds36_traits['site name'] == ds36_traits['site name'].unique()[i], 'Longitude'] = lon[i]
    ds36_traits.loc[ds36_traits['site name'] == ds36_traits['site name'].unique()[i], 'Site ID'] = 'Site#'+str(k)
    k = k+1
    
for i in range(len(ds36_traits['Latin Name'].unique())):
    ds36_traits.loc[ds36_traits['Latin Name'] == ds36_traits['Latin Name'].unique()[i], 'PFT'] = pfts[i]
    
ds36_traits['Dataset ID'] = 'Dataset#36'
ds36_traits['LMA'] = ds36_traits['LMA']/10000

ds36_traits['Dataset Name'] = 'leaf traits and leaf spectroscopy of east Asian forest tree species'
ds36_traits["Foreoptic type"] = "Contact probe"
ds36_traits["Instruments"] = "ASD FieldSpec Pro FR"
ds36_traits["Reference"] = "(Nakaji et al., 2019)"
ds36_traits["Link to original dataset"] = "https://doi.org/10.1016/j.rse.2019.111381"
ds36_traits["DOI"] = "https://doi.org/10.1016/j.rse.2019.111381"
ds36_traits = ds36_traits[["Dataset ID","Dataset Name","Site ID","Sample date","Latin Name","PFT","Latitude",
                           "Longitude","LMA","Foreoptic type", "Instruments","Reference",
                           "Link to original dataset","DOI"]]

ds36_reflectance = ds36_reflectance.loc[:,"448":"2405"]
ds36_reflectance = do_resample(ds36_reflectance)
ds36 = pd.concat([ds36_reflectance, ds36_traits],axis = 1)
# ds36

### 37. Leaf reflectance plant functional gradient IFGG/KIT

In [39]:
df = pd.read_csv('../1_datasets/0_original/37_Leaf reflectance plant functional gradient IFGG_KIT/leaf_kit_plant_functional_gradient.csv',encoding= 'unicode_escape')
df = df.iloc[1:,:]

ds37_reflectance = df.loc[:,'400':]
ds37_traits = df.loc[:,:'LDMC (g/g)']

ds37_traits['SLA (g/cm²)'] = ds37_traits['SLA (g/cm²)'].astype(float)
ds37_traits['Chlorophyll content (µg/cm²)'] = ds37_traits['Chlorophyll content (µg/cm²)'].astype(float)
ds37_traits['Carotenoid content (µg/cm²)'] = ds37_traits['Carotenoid content (µg/cm²)'].astype(float)
ds37_traits['LWC (mg/cm²)'] = ds37_traits['LWC (mg/cm²)'].astype(float)


ds37_traits['Dataset ID'] = 'Dataset#37'
ds37_traits['Site ID'] = 'Site#104'
ds37_traits['Sample date'] = pd.to_datetime(ds37_traits['timestamp'])
ds37_traits['Sample date'] = ds37_traits['Sample date'].dt.strftime('%Y%m%d')

ds37_traits['English Name'] = ds37_traits['species']
ds37_traits['PFT'] = 'Grasslands'
ds37_traits['Latitude'] = 49.0129048
ds37_traits['Longitude'] = 8.4190334


ds37_traits['Chla+b'] = ds37_traits['Chlorophyll content (µg/cm²)']
ds37_traits['Ccar'] = ds37_traits['Carotenoid content (µg/cm²)']
ds37_traits['EWT'] = ds37_traits['LWC (mg/cm²)']/1000
ds37_traits['LMA'] = 1/ds37_traits['SLA (g/cm²)']
ds37_traits['English Name'] = ds37_traits['species']

ds37_traits['Dataset Name'] = 'Leaf reflectance plant functional gradient IFGG/KIT'
ds37_traits["Foreoptic type"] = "Leaf clip"
ds37_traits["Instruments"] = "ASD FieldSpec 3"
ds37_traits["Reference"] = "(Kattenborn et al., 2017)"
ds37_traits["Link to original dataset"] = "https://data.ecosis.org/dataset/leaf-reflectance-plant-functional-gradient-ifgg-kit"
ds37_traits["DOI"] = "https://doi.org/doi:10.21232/dx1z-qz48"
ds37_traits = ds37_traits[["Dataset ID","Dataset Name","Site ID","Sample date","English Name",
                           "PFT","Latitude","Longitude", "Chla+b","Ccar","EWT","LMA","Foreoptic type",
                           "Instruments","Reference","Link to original dataset","DOI"]]

ds37_traits.reset_index(drop = True, inplace = True)

ds37_reflectance = ds37_reflectance.loc[:,"450":"2400"]
ds37_reflectance = do_resample(ds37_reflectance)
ds37 = pd.concat([ds37_reflectance, ds37_traits],axis = 1)
# ds37

### Datasets compliation

In [40]:
dataset = pd.concat([ds1,ds2,ds3,ds4,ds5,ds6,ds7,ds8,ds9,ds10,ds11,ds12,ds13,ds14,ds15,ds16,ds17,ds18,ds19,
                     ds20,ds21,ds22,ds23,ds24,ds25,ds26,ds27,ds28,ds29,ds30,ds31,ds32,ds33,ds34,ds35,ds36,
                     ds37],axis = 0)
dataset = dataset[(dataset["Chla+b"]>0)|(dataset["Ccar"]>0)|(dataset["EWT"]>0)|(dataset["LMA"]>0)]
dataset["EWT"] = dataset["EWT"]*10000
dataset["LMA"] = dataset["LMA"]*10000
dataset.reset_index(drop = True, inplace = True)
dataset.to_csv('../1_datasets/Paired leaf traits and leaf spectra dataset.csv',index = False)